In [1]:
%pip install google-cloud-aiplatform --upgrade
!pip install google-auth
%pip install openai
!pip install neo4j

   ---------------------------------------- 0.0/4.8 MB ? eta -:--:--
   ---- ----------------------------------- 0.5/4.8 MB 10.5 MB/s eta 0:00:01
   ------- -------------------------------- 0.9/4.8 MB 11.0 MB/s eta 0:00:01
   ------------ --------------------------- 1.5/4.8 MB 10.4 MB/s eta 0:00:01
   ---------------- ----------------------- 2.0/4.8 MB 11.5 MB/s eta 0:00:01
   --------------------- ------------------ 2.5/4.8 MB 12.5 MB/s eta 0:00:01
   ------------------------- -------------- 3.0/4.8 MB 12.1 MB/s eta 0:00:01
   -------------------------- ------------- 3.2/4.8 MB 11.2 MB/s eta 0:00:01
   ----------------------------- ---------- 3.6/4.8 MB 10.9 MB/s eta 0:00:01
   --------------------------------- ------ 4.1/4.8 MB 10.8 MB/s eta 0:00:01
   ------------------------------------- -- 4.6/4.8 MB 10.8 MB/s eta 0:00:01
   ---------------------------------------- 4.8/4.8 MB 10.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/138.3 kB ? eta -:--:--
   ----------

   ---------------------------------------- 0.0/311.6 kB ? eta -:--:--
   ----------- ---------------------------- 92.2/311.6 kB 5.1 MB/s eta 0:00:01
   ------------------------------------ --- 286.7/311.6 kB 4.5 MB/s eta 0:00:01
   ---------------------------------------- 311.6/311.6 kB 3.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/75.6 kB ? eta -:--:--
   ---------------------------------------- 75.6/75.6 kB 4.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/77.9 kB ? eta -:--:--
   ---------------------------------------- 77.9/77.9 kB 2.2 MB/s eta 0:00:00
Using cached h11-0.14.0-py3-none-any.whl (58 kB)
Note: you may need to restart the kernel to use updated packages.
     ---------------------------------------- 0.0/203.0 kB ? eta -:--:--
     ---------------------------------------- 0.0/203.0 kB ? eta -:--:--
     -- ------------------------------------- 10.2/203.0 kB ? eta -:--:--
     ----- ------------------------------- 30.7/203.0 kB 33

In [2]:
# Import Vertex AI library
from google.cloud import aiplatform

# Import service account 
from google.oauth2 import service_account

In [3]:
import vertexai

project_id = 'resumeanz'
location = 'us-central1'
credentials = service_account.Credentials.from_service_account_file("resumeanz-db0c67277823.json")

vertexai.init(project='resumeanz',credentials=credentials)

aiplatform.TabularDataset.list()


[]

In [57]:
person_prompt_tpl="""From the Resume text for a job aspirant below, extract Entities strictly as instructed below
1. First, look for the Person Entity type in the text and extract the needed information defined below:
   `id` property of each entity must be alphanumeric and must be unique among the entities. You will be referring this property to define the relationship between entities. NEVER create new entity types that aren't mentioned below. Document must be summarized and stored inside Person entity under `description` property
    Entity Types:
    label:'Person',id:string,role:string,description:string //Person Node
2. Description property should be a crisp text summary and MUST NOT be more than 100 characters
3. If you cannot find any information on the entities & relationships above, it is okay to return empty value. DO NOT create fictious data
4. Do NOT create duplicate entities
5. Restrict yourself to extract only Person information. No Position, Company, Education or Skill information should be focussed.
6. NEVER Impute missing values
Example Output JSON:
{"entities": [{"label":"Person","id":"person1","role":"Prompt Developer","description":"Prompt Developer with more than 30 years of LLM experience"}]}

Question: Now, extract the Person for the text below -
$ctext

Answer:
"""

In [58]:
postion_prompt_tpl="""From the Resume text for a job aspirant below, extract Entities & relationships strictly as instructed below
1. First, look for Position & Company types in the text and extract information in comma-separated format. Position Entity denotes the Person's previous or current job. Company node is the Company where they held that position.
   `id` property of each entity must be alphanumeric and must be unique among the entities. You will be referring this property to define the relationship between entities. NEVER create new entity types that aren't mentioned below. You will have to generate as many entities as needed as per the types below:
    Entity Types:
    label:'Position',id:string,title:string,location:string,startDate:string,endDate:string,url:string //Position Node
    label:'Company',id:string,name:string //Company Node
2. Next generate each relationships as triples of head, relationship and tail. To refer the head and tail entity, use their respective `id` property. NEVER create new Relationship types that aren't mentioned below:
    Relationship definition:
    position|AT_COMPANY|company //Ensure this is a string in the generated output
3. If you cannot find any information on the entities & relationships above, it is okay to return empty value. DO NOT create fictious data
4. Do NOT create duplicate entities.
5. No Education or Skill information should be extracted.
6. DO NOT MISS out any Position or Company related information
7. NEVER Impute missing values
 Example Output JSON:
{"entities": [{"label":"Position","id":"position1","title":"Software Engineer","location":"Singapore",startDate:"2021-01-01",endDate:"present"},{"label":"Position","id":"position2","title":"Senior Software Engineer","location":"Mars",startDate:"2020-01-01",endDate:"2020-12-31"},{label:"Company",id:"company1",name:"Neo4j Singapore Pte Ltd"},{"label":"Company","id":"company2","name":"Neo4j Mars Inc"}],"relationships": ["position1|AT_COMPANY|company1","position2|AT_COMPANY|company2"]}

Question: Now, extract entities & relationships as mentioned above for the text below -
$ctext

Answer:
"""

In [59]:
skill_prompt_tpl="""From the Resume text below, extract Entities strictly as instructed below
1. Look for prominent Skill Entities in the text. The`id` property of each entity must be alphanumeric and must be unique among the entities. NEVER create new entity types that aren't mentioned below:
    Entity Definition:
    label:'Skill',id:string,name:string,level:string //Skill Node
2. NEVER Impute missing values
3. If you do not find any level information: assume it as `expert` if the experience in that skill is more than 5 years, `intermediate` for 2-5 years and `beginner` otherwise.
Example Output Format:
{"entities": [{"label":"Skill","id":"skill1","name":"Neo4j","level":"expert"},{"label":"Skill","id":"skill2","name":"Pytorch","level":"expert"}]}

Question: Now, extract entities as mentioned above for the text below -
$ctext

Answer:
"""

In [60]:
edu_prompt_tpl="""From the Resume text for a job aspirant below, extract Entities strictly as instructed below
1. Look for Education entity type and generate the information defined below:
   `id` property of each entity must be alphanumeric and must be unique among the entities. You will be referring this property to define the relationship between entities. NEVER create other entity types that aren't mentioned below. You will have to generate as many entities as needed as per the types below:
    Entity Definition:
    label:'Education',id:string,degree:string,university:string,graduationDate:string,score:string,url:string //Education Node
2. If you cannot find any information on the entities above, it is okay to return empty value. DO NOT create fictious data
3. Do NOT create duplicate entities or properties
4. Strictly extract only Education. No Skill or other Entities should be extracted
5. DO NOT MISS out any Education related entity
6. NEVER Impute missing values
Output JSON (Strict):
{"entities": [{"label":"Education","id":"education1","degree":"Bachelor of Science","graduationDate":"May 2022","score":"0.0"}]}

Question: Now, extract Education information as mentioned above for the text below -
$ctext

Answer:
"""

In [61]:
from vertexai.preview.language_models import TextGenerationModel

def run_text_model(
    project_id: str,
    model_name: str,
    temperature: float,
    max_decode_steps: int,
    top_p: float,
    top_k: int,
    prompt: str,
    location: str = location,
    tuned_model_name: str = "",
    ) :
    """Text Completion Use a Large Language Model."""
    vertexai.init(project='resumeanz',credentials=credentials)
    model = TextGenerationModel.from_pretrained(model_name)
    if tuned_model_name:
      model = model.get_tuned_model(tuned_model_name)
    response = model.predict(
        prompt,
        temperature=temperature,
        max_output_tokens=max_decode_steps,
        top_k=top_k,
        top_p=top_p,)
    return response.text


def extract_entities_relationships(prompt, tuned_model_name):
    try:
        res = run_text_model(project_id, "text-bison@001", 0, 1024, 0.8, 40, prompt, location, tuned_model_name)
        return res
    except Exception as e:
        print(e)


import re
def clean_text(text):
    return re.sub(r'[^\x00-\x7F]+',' ', text)

In [62]:
from string import Template
import json

sample_que = """Developer <span class="hl">Developer</span> Developer - TATA CONSULTANTCY SERVICE Batavia, OH Relevant course work† Database Systems, Database Administration, Database Security & Auditing, Computer Security,Computer Networks, Programming & Software Development, IT, Information Security Concept & Admin,† IT System Acquisition & Integration, Advanced Web Development, and Ethical Hacking: Network Security & Pen Testing. Work Experience Developer TATA CONSULTANTCY SERVICE June 2016 to Present MRM (Government of ME, RI, MS) Developer†††† Working with various technologies such as Java, JSP, JSF, DB2(SQL), LDAP, BIRT report, Jazz version control, Squirrel SQL client, Hibernate, CSS, Linux, and Windows. Work as part of a team that provide support to enterprise applications. Perform miscellaneous support activities as requested by Management. Perform in-depth research and identify sources of production issues.†† SPLUNK Developer† Supporting the Splunk Operational environment for Business Solutions Unit aiming to support overall business infrastructure. Developing Splunk Queries to generate the report, monitoring, and analyzing machine generated big data for server that has been using for onsite and offshore team. Working with Splunk' premium apps such as ITSI, creating services, KPI, and glass tables. Developing app with custom dashboard with front- end ability and advanced XML to serve Business Solution unit' needs. Had in-house app presented at Splunk's .Conf Conference (2016). Help planning, prioritizing and executing development activities. Developer ( front end) intern TOMORROW PICTURES INC - Atlanta, GA April 2015 to January 2016 Assist web development team with multiple front end web technologies and involved in web technologies such as Node.js, express, json, gulp.js, jade, sass, html5, css3, bootstrap, WordPress.†Testing (manually), version control (GitHub), mock up design and ideas Education MASTER OF SCIENCE IN INFORMATION TECHNOLOGY in INFOTMATION TECHNOLOGY KENNESAW STATE UNIVERSITY - Kennesaw, GA August 2012 to May 2015 MASTER OF BUSINESS ADMINISTRATION in INTERNATIONAL BUSINESS AMERICAN INTER CONTINENTAL UNIVERSITY ATLANTA November 2003 to December 2005 BACHELOR OF ARTS in PUBLIC RELATIONS THE UNIVERSITY OF THAI CHAMBER OF COMMERCE - BANGKOK, TH June 1997 to May 2001 Skills Db2 (2 years), front end (2 years), Java (2 years), Linux (2 years), Splunk (2 years), SQL (3 years) Certifications/Licenses Splunk Certified Power User V6.3 August 2016 to Present CERT-112626 Splunk Certified Power User V6.x May 2017 to Present CERT-168138 Splunk Certified User V6.x May 2017 to Present CERT -181476 Driver's License Additional Information Skills† ∑††††SQL, PL/SQL, Knowledge of Data Modeling, Experience on Oracle database/RDBMS.† ∑††††††††Database experience on Oracle, DB2, SQL Sever, MongoDB, and MySQL.† ∑††††††††Knowledge of tools including Splunk, tableau, and wireshark.† ∑††††††††Knowledge of SCRUM/AGILE and WATERFALL methodologies.† ∑††††††††Web technology included: HTML5, CSS3, XML, JSON, JavaScript, node.js, NPM, GIT, express.js, jQuery, Angular, Bootstrap, and Restful API.† ∑††††††††Working Knowledge in JAVA, J2EE, and PHP.† Operating system Experience included: Windows, Mac OS, Linux (Ubuntu, Mint, Kali)††"""
prompts = [person_prompt_tpl, postion_prompt_tpl, skill_prompt_tpl, edu_prompt_tpl]
results = {"entities": [], "relationships": []}

for p in prompts:
    _prompt = Template(p).substitute(ctext=clean_text(sample_que))
    _extraction = extract_entities_relationships(_prompt, '')
    if 'Answer:\n' in _extraction:
        _extraction = _extraction.split('Answer:\n ')[1]
    if _extraction.strip() == '':
        continue
    try:
        _extraction = json.loads(_extraction.replace("\'", "'").replace('`', ''))
    except json.JSONDecodeError:
        # print(_extraction)
        #Temp hack to ignore Skills cut off by token limitation
        _extraction = _extraction[:_extraction.rfind("}")+1] + ']}'
        _extraction = json.loads(_extraction.replace("\'", "'"))
    results["entities"].extend(_extraction["entities"])
    if "relationships" in _extraction:
        results["relationships"].extend(_extraction["relationships"])


person_id = results["entities"][0]["id"]
for e in results["entities"][1:]:
    if e['label'] == 'Position':
        results["relationships"].append(f"{person_id}|HAS_POSITION|{e['id']}")
    if e['label'] == 'Skill':
        results["relationships"].append(f"{person_id}|HAS_SKILL|{e['id']}")
    if e['label'] == 'Education':
        results["relationships"].append(f"{person_id}|HAS_EDUCATION|{e['id']}")

In [63]:
results

{'entities': [{'label': 'Person',
   'id': 'person1',
   'role': 'Developer',
   'description': 'Developer with 10+ years of experience in IT industry.'},
  {'label': 'Position',
   'id': 'position1',
   'title': 'Developer',
   'location': 'Batavia, OH',
   'startDate': '2016-06-01',
   'endDate': 'present'},
  {'label': 'Company', 'id': 'company1', 'name': 'TATA CONSULTANTCY SERVICE'},
  {'label': 'Skill', 'id': 'skill1', 'name': 'SQL', 'level': 'expert'},
  {'label': 'Skill', 'id': 'skill2', 'name': 'Java', 'level': 'expert'},
  {'label': 'Skill', 'id': 'skill3', 'name': 'Linux', 'level': 'expert'},
  {'label': 'Skill', 'id': 'skill4', 'name': 'Splunk', 'level': 'expert'},
  {'label': 'Skill', 'id': 'skill5', 'name': 'front end', 'level': 'expert'},
  {'label': 'Skill', 'id': 'skill6', 'name': 'Db2', 'level': 'expert'},
  {'label': 'Skill', 'id': 'skill7', 'name': 'HTML5', 'level': 'expert'},
  {'label': 'Skill', 'id': 'skill8', 'name': 'CSS3', 'level': 'expert'},
  {'label': 'Skill

In [64]:


def get_prop_str(prop_dict, _id):
    s = []
    for key, val in prop_dict.items():
      if key != 'label' and key != 'id':
         s.append(_id+"."+key+' = "'+str(val).replace('\"', '"').replace('"', '\"')+'"')
    return ' ON CREATE SET ' + ','.join(s)

def get_cypher_compliant_var(_id):
    s = "_"+ re.sub(r'[\W_]', '', _id).lower() 
    return s[:20]

def generate_cypher(file_name, in_json):
    e_map = {}
    e_stmt = []
    r_stmt = []
    e_stmt_tpl = Template("($id:$label{id:'$key'})")
    r_stmt_tpl = Template("""
      MATCH $src
      MATCH $tgt
      MERGE ($src_id)-[:$rel]->($tgt_id)
    """)
    for obj in in_json:
      for j in obj['entities']:
          props = ''
          label = j['label']
          id = ''
          if label == 'Person':
            id = 'p'+str(file_name)
          elif label == 'Position':
            c = j['id'].replace('position', '_')
            id = f'j{str(file_name)}{c}'
          elif label == 'Education':
            c = j['id'].replace('education', '_')
            id = f'e{str(file_name)}{c}'
          else:
            id = get_cypher_compliant_var(j['name'])
          if label in ['Person', 'Position', 'Education', 'Skill', 'Company']:
            varname = get_cypher_compliant_var(j['id'])
            stmt = e_stmt_tpl.substitute(id=varname, label=label, key=id)
            e_map[varname] = stmt
            e_stmt.append('MERGE '+ stmt + get_prop_str(j, varname))

      for st in obj['relationships']:
          rels = st.split("|")
          src_id = get_cypher_compliant_var(rels[0].strip())
          rel = rels[1].strip()
          if rel in ['HAS_SKILL', 'HAS_EDUCATION', 'AT_COMPANY', 'HAS_POSITION']: #we ignore other relationships
            tgt_id = get_cypher_compliant_var(rels[2].strip())
            stmt = r_stmt_tpl.substitute(
              src_id=src_id, tgt_id=tgt_id, src=e_map[src_id], tgt=e_map[tgt_id], rel=rel)
            r_stmt.append(stmt)

    return e_stmt, r_stmt

In [65]:
ent_cyp, rel_cyp = generate_cypher('my_cv', [results])

print(ent_cyp, rel_cyp)

['MERGE (_person1:Person{id:\'pmy_cv\'}) ON CREATE SET _person1.role = "Developer",_person1.description = "Developer with 10+ years of experience in IT industry."', 'MERGE (_position1:Position{id:\'jmy_cv_1\'}) ON CREATE SET _position1.title = "Developer",_position1.location = "Batavia, OH",_position1.startDate = "2016-06-01",_position1.endDate = "present"', 'MERGE (_company1:Company{id:\'_tataconsultantcyser\'}) ON CREATE SET _company1.name = "TATA CONSULTANTCY SERVICE"', 'MERGE (_skill1:Skill{id:\'_sql\'}) ON CREATE SET _skill1.name = "SQL",_skill1.level = "expert"', 'MERGE (_skill2:Skill{id:\'_java\'}) ON CREATE SET _skill2.name = "Java",_skill2.level = "expert"', 'MERGE (_skill3:Skill{id:\'_linux\'}) ON CREATE SET _skill3.name = "Linux",_skill3.level = "expert"', 'MERGE (_skill4:Skill{id:\'_splunk\'}) ON CREATE SET _skill4.name = "Splunk",_skill4.level = "expert"', 'MERGE (_skill5:Skill{id:\'_frontend\'}) ON CREATE SET _skill5.name = "front end",_skill5.level = "expert"', 'MERGE (_

In [5]:
from neo4j import GraphDatabase
connectionUrl = "neo4j+s://fe656664.databases.neo4j.io"
username = "neo4j"
password = "nY2z39snwB9Bq1s1iRLih9XBH8-C5gI4R46AM4R89X8"
driver = GraphDatabase.driver(connectionUrl, auth=(username, password))
driver.verify_connectivity()



In [67]:
import pandas as pd
def run_query(query, params={}):
    with driver.session() as session:
        result = session.run(query, params)
        return pd.DataFrame([r.values() for r in result], columns=result.keys())
    

run_query('CREATE CONSTRAINT unique_person_id IF NOT EXISTS FOR (n:Person) REQUIRE (n.id) IS UNIQUE')
run_query('CREATE CONSTRAINT unique_position_id IF NOT EXISTS FOR (n:Position) REQUIRE (n.id) IS UNIQUE')
run_query('CREATE CONSTRAINT unique_skill_id IF NOT EXISTS FOR (n:Skill) REQUIRE n.id IS UNIQUE')
run_query('CREATE CONSTRAINT unique_education_id IF NOT EXISTS FOR (n:Education) REQUIRE n.id IS UNIQUE')
run_query('CREATE CONSTRAINT unique_company_id IF NOT EXISTS FOR (n:Company) REQUIRE n.id IS UNIQUE')

""


In [68]:
%%time
for e in ent_cyp:
    run_query(e)

CPU times: total: 62.5 ms
Wall time: 5.51 s


In [69]:
%%time
for r in rel_cyp:
    run_query(r)

CPU times: total: 46.9 ms
Wall time: 5.9 s


In [70]:
import glob
from string import Template
import json
from pathlib import Path
import re


def run_pipeline(start=0, count=1):
    txt_files = glob.glob("data/*.txt")[start:count]
    print(f"Running pipeline for {len(txt_files)} files")
    failed_files = process_pipeline(txt_files)
    print(failed_files)
    return failed_files

def process_pipeline(files):
    failed_files = []
    i = 0
    for f in files:
        i += 1
        try:
            with open(f, 'r', encoding='utf-8', errors='ignore') as file:
                print(f"  {f}: Reading File No. ({i})")
                data = file.read().rstrip()
                text = data
                print(f"    {f}: Extracting Entities & Relationships")
                results = run_extraction(f, text)
                print(f"    {f}: Generating Cypher")
                ent_cyp, rel_cyp = generate_cypher(Path(f).stem, results)
                print(f"    {f}: Ingesting Entities")
                for e in ent_cyp:
                    run_query(e)
                print(f"    {f}: Ingesting Relationships")
                for r in rel_cyp:
                    run_query(r)
                print(f"    {f}: Processing DONE")
        except Exception as e:
            print(f"    {f}: Processing Failed with exception {e}")
            failed_files.append(f)
    return failed_files
        
from timeit import default_timer as timer
def run_extraction(f, text):
    start = timer()
    prompts = [person_prompt_tpl, postion_prompt_tpl, skill_prompt_tpl, edu_prompt_tpl]
    results = {"entities": [], "relationships": []}
    for p in prompts:
        _prompt = Template(p).substitute(ctext=text)
        _extraction = extract_entities_relationships(_prompt, '')
        if 'Answer:\n' in _extraction:
            _extraction = _extraction.split('Answer:\n ')[1]
        if _extraction.strip() == '':
            continue
        try:
            _extraction = json.loads(_extraction.replace("\'", "'"))
        except json.JSONDecodeError:
            #Temp hack to ignore Skills cut off by token limitation
            _extraction = _extraction[:_extraction.rfind("}")+1] + ']}'
            _extraction = json.loads(_extraction.replace("\'", "'"))
        for entity in _extraction["entities"]:
            if entity["label"] == "Person":
                description = entity["description"]
        results["entities"].extend(_extraction["entities"])
        if "relationships" in _extraction:
            results["relationships"].extend(_extraction["relationships"])
    person_id = results["entities"][0]["id"]
    for e in results["entities"][1:]:
        if e['label'] == 'Position':
            results["relationships"].append(f"{person_id}|HAS_POSITION|{e['id']}")
        if e['label'] == 'Skill':
            results["relationships"].append(f"{person_id}|HAS_SKILL|{e['id']}")
        if e['label'] == 'Education':
            results["relationships"].append(f"{person_id}|HAS_EDUCATION|{e['id']}")
    end = timer()
    elapsed = (end-start)
    print(f"    {f}: Entity Extraction took {elapsed}secs")
    return [results]

In [ ]:
failed_files = run_pipeline(0, 1000)


Running pipeline for 1000 files
  data\05499.txt: Reading File No. (1)
    data\05499.txt: Extracting Entities & Relationships
    data\05499.txt: Entity Extraction took 6.53550019999966secs
    data\05499.txt: Generating Cypher
    data\05499.txt: Ingesting Entities
    data\05499.txt: Ingesting Relationships
    data\05499.txt: Processing DONE
  data\05500.txt: Reading File No. (2)
    data\05500.txt: Extracting Entities & Relationships
    data\05500.txt: Entity Extraction took 6.030318099999931secs
    data\05500.txt: Generating Cypher
    data\05500.txt: Ingesting Entities
    data\05500.txt: Ingesting Relationships
    data\05500.txt: Processing DONE
  data\05501.txt: Reading File No. (3)
    data\05501.txt: Extracting Entities & Relationships
    data\05501.txt: Entity Extraction took 8.152375099998608secs
    data\05501.txt: Generating Cypher
    data\05501.txt: Ingesting Entities
    data\05501.txt: Ingesting Relationships
    data\05501.txt: Processing DONE
  data\05502.txt: 

    data\05525.txt: Entity Extraction took 9.960429500000828secs
    data\05525.txt: Generating Cypher
    data\05525.txt: Ingesting Entities
    data\05525.txt: Ingesting Relationships
    data\05525.txt: Processing DONE
  data\05526.txt: Reading File No. (28)
    data\05526.txt: Extracting Entities & Relationships
    data\05526.txt: Entity Extraction took 5.882630100000824secs
    data\05526.txt: Generating Cypher
    data\05526.txt: Ingesting Entities
    data\05526.txt: Ingesting Relationships
    data\05526.txt: Processing DONE
  data\05527.txt: Reading File No. (29)
    data\05527.txt: Extracting Entities & Relationships
    data\05527.txt: Entity Extraction took 3.9104271000014705secs
    data\05527.txt: Generating Cypher
    data\05527.txt: Ingesting Entities
    data\05527.txt: Ingesting Relationships
    data\05527.txt: Processing DONE
  data\05528.txt: Reading File No. (30)
    data\05528.txt: Extracting Entities & Relationships
    data\05528.txt: Entity Extraction took 6.

    data\05551.txt: Entity Extraction took 9.042477700000745secs
    data\05551.txt: Generating Cypher
    data\05551.txt: Ingesting Entities
    data\05551.txt: Ingesting Relationships
    data\05551.txt: Processing DONE
  data\05552.txt: Reading File No. (54)
    data\05552.txt: Extracting Entities & Relationships
    data\05552.txt: Entity Extraction took 6.1477150000009715secs
    data\05552.txt: Generating Cypher
    data\05552.txt: Ingesting Entities
    data\05552.txt: Ingesting Relationships
    data\05552.txt: Processing DONE
  data\05553.txt: Reading File No. (55)
    data\05553.txt: Extracting Entities & Relationships
    data\05553.txt: Entity Extraction took 4.627639899999849secs
    data\05553.txt: Generating Cypher
    data\05553.txt: Ingesting Entities
    data\05553.txt: Ingesting Relationships
    data\05553.txt: Processing DONE
  data\05554.txt: Reading File No. (56)
    data\05554.txt: Extracting Entities & Relationships
    data\05554.txt: Entity Extraction took 5.

    data\05577.txt: Entity Extraction took 7.611068299998806secs
    data\05577.txt: Generating Cypher
    data\05577.txt: Ingesting Entities
    data\05577.txt: Ingesting Relationships
    data\05577.txt: Processing DONE
  data\05578.txt: Reading File No. (80)
    data\05578.txt: Extracting Entities & Relationships
    data\05578.txt: Entity Extraction took 6.793208499999309secs
    data\05578.txt: Generating Cypher
    data\05578.txt: Ingesting Entities
    data\05578.txt: Ingesting Relationships
    data\05578.txt: Processing DONE
  data\05579.txt: Reading File No. (81)
    data\05579.txt: Extracting Entities & Relationships
    data\05579.txt: Entity Extraction took 12.738823399999092secs
    data\05579.txt: Generating Cypher
    data\05579.txt: Ingesting Entities
    data\05579.txt: Ingesting Relationships
    data\05579.txt: Processing DONE
  data\05580.txt: Reading File No. (82)
    data\05580.txt: Extracting Entities & Relationships
    data\05580.txt: Entity Extraction took 13

    data\05603.txt: Entity Extraction took 7.374985499998729secs
    data\05603.txt: Generating Cypher
    data\05603.txt: Ingesting Entities
    data\05603.txt: Ingesting Relationships
    data\05603.txt: Processing DONE
  data\05604.txt: Reading File No. (106)
    data\05604.txt: Extracting Entities & Relationships
    data\05604.txt: Entity Extraction took 10.980534899999839secs
    data\05604.txt: Generating Cypher
    data\05604.txt: Ingesting Entities
    data\05604.txt: Ingesting Relationships
    data\05604.txt: Processing DONE
  data\05605.txt: Reading File No. (107)
    data\05605.txt: Extracting Entities & Relationships
    data\05605.txt: Entity Extraction took 6.913010499998563secs
    data\05605.txt: Generating Cypher
    data\05605.txt: Ingesting Entities
    data\05605.txt: Ingesting Relationships
    data\05605.txt: Processing DONE
  data\05606.txt: Reading File No. (108)
    data\05606.txt: Extracting Entities & Relationships
    data\05606.txt: Entity Extraction took

...

    data\05655.txt: Processing DONE
  data\05656.txt: Reading File No. (158)
    data\05656.txt: Extracting Entities & Relationships
    data\05656.txt: Entity Extraction took 9.291780699999435secs
    data\05656.txt: Generating Cypher
    data\05656.txt: Ingesting Entities
    data\05656.txt: Ingesting Relationships
    data\05656.txt: Processing DONE
  data\05657.txt: Reading File No. (159)
    data\05657.txt: Extracting Entities & Relationships
    data\05657.txt: Entity Extraction took 6.194659400000091secs
    data\05657.txt: Generating Cypher
    data\05657.txt: Ingesting Entities
    data\05657.txt: Ingesting Relationships
    data\05657.txt: Processing DONE
  data\05658.txt: Reading File No. (160)
    data\05658.txt: Extracting Entities & Relationships
    data\05658.txt: Entity Extraction took 11.519543299998986secs
    data\05658.txt: Generating Cypher
    data\05658.txt: Ingesting Entities
    data\05658.txt: Ingesting Relationships
    data\05658.txt: Processing DONE
  data

    data\05681.txt: Ingesting Relationships
    data\05681.txt: Processing DONE
  data\05682.txt: Reading File No. (184)
    data\05682.txt: Extracting Entities & Relationships
    data\05682.txt: Entity Extraction took 5.865416299999197secs
    data\05682.txt: Generating Cypher
    data\05682.txt: Ingesting Entities
    data\05682.txt: Ingesting Relationships
    data\05682.txt: Processing DONE
  data\05683.txt: Reading File No. (185)
    data\05683.txt: Extracting Entities & Relationships
    data\05683.txt: Entity Extraction took 10.322757600000841secs
    data\05683.txt: Generating Cypher
    data\05683.txt: Ingesting Entities
    data\05683.txt: Ingesting Relationships
    data\05683.txt: Processing DONE
  data\05684.txt: Reading File No. (186)
    data\05684.txt: Extracting Entities & Relationships
    data\05684.txt: Entity Extraction took 11.74977349999972secs
    data\05684.txt: Generating Cypher
    data\05684.txt: Ingesting Entities
    data\05684.txt: Ingesting Relationship

    data\05707.txt: Ingesting Relationships
    data\05707.txt: Processing DONE
  data\05708.txt: Reading File No. (210)
    data\05708.txt: Extracting Entities & Relationships
    data\05708.txt: Processing Failed with exception Expecting value: line 1 column 1 (char 0)
  data\05709.txt: Reading File No. (211)
    data\05709.txt: Extracting Entities & Relationships
    data\05709.txt: Processing Failed with exception Expecting value: line 1 column 1 (char 0)
  data\05710.txt: Reading File No. (212)
    data\05710.txt: Extracting Entities & Relationships
    data\05710.txt: Entity Extraction took 14.030056800000239secs
    data\05710.txt: Generating Cypher
    data\05710.txt: Ingesting Entities
    data\05710.txt: Ingesting Relationships
    data\05710.txt: Processing DONE
  data\05711.txt: Reading File No. (213)
    data\05711.txt: Extracting Entities & Relationships
    data\05711.txt: Entity Extraction took 17.06450500000028secs
    data\05711.txt: Generating Cypher
    data\05711.t

    data\05734.txt: Entity Extraction took 5.374793099999806secs
    data\05734.txt: Generating Cypher
    data\05734.txt: Ingesting Entities
    data\05734.txt: Ingesting Relationships
    data\05734.txt: Processing DONE
  data\05735.txt: Reading File No. (237)
    data\05735.txt: Extracting Entities & Relationships
    data\05735.txt: Entity Extraction took 10.14190569999846secs
    data\05735.txt: Generating Cypher
    data\05735.txt: Ingesting Entities
    data\05735.txt: Ingesting Relationships
    data\05735.txt: Processing DONE
  data\05736.txt: Reading File No. (238)
    data\05736.txt: Extracting Entities & Relationships
    data\05736.txt: Entity Extraction took 14.09348990000035secs
    data\05736.txt: Generating Cypher
    data\05736.txt: Ingesting Entities
    data\05736.txt: Ingesting Relationships
    data\05736.txt: Processing DONE
  data\05737.txt: Reading File No. (239)
    data\05737.txt: Extracting Entities & Relationships
    data\05737.txt: Entity Extraction took 

    data\05760.txt: Entity Extraction took 7.775100399998337secs
    data\05760.txt: Generating Cypher
    data\05760.txt: Ingesting Entities
    data\05760.txt: Ingesting Relationships
    data\05760.txt: Processing DONE
  data\05761.txt: Reading File No. (263)
    data\05761.txt: Extracting Entities & Relationships
    data\05761.txt: Entity Extraction took 14.379607099999703secs
    data\05761.txt: Generating Cypher
    data\05761.txt: Ingesting Entities
    data\05761.txt: Ingesting Relationships
    data\05761.txt: Processing DONE
  data\05762.txt: Reading File No. (264)
    data\05762.txt: Extracting Entities & Relationships
    data\05762.txt: Entity Extraction took 7.576363200001651secs
    data\05762.txt: Generating Cypher
    data\05762.txt: Ingesting Entities
    data\05762.txt: Ingesting Relationships
    data\05762.txt: Processing DONE
  data\05763.txt: Reading File No. (265)
    data\05763.txt: Extracting Entities & Relationships
    data\05763.txt: Entity Extraction took

In [6]:
from openai import OpenAI
client = OpenAI(api_key='')



def generate_openai_embeddings(text):

    response = client.embeddings.create(
            model="text-embedding-3-small",
            input=text,
            encoding_format="float"
            )
    return response.data[0].embedding

def fetch_person_descriptions():
    with driver.session() as session:
        result = session.run("MATCH (p:Person) WHERE p.description IS NOT NULL RETURN p.id AS id, p.description AS description")
        return [(record["id"], record["description"]) for record in result]

def get_text_embedding(text):
    embeddings = generate_openai_embeddings(text)
    return embeddings
 
def update_person_embedding(node_id, embedding):
    with driver.session() as session:
        session.run("MATCH (p:Person {id: $id}) SET p.embedding = $embedding", id=node_id, embedding=embedding)

In [7]:
import time
 
persons = fetch_person_descriptions()

count = 0
 
for person_id, description in persons:   

    embedding = get_text_embedding(description)    

    update_person_embedding(person_id, embedding)

In [51]:
failed_files

['data/09558.txt',
 'data/10021.txt',
 'data/09941.txt',
 'data/05982.txt',
 'data/05941.txt',
 'data/07415.txt',
 'data/06086.txt',
 'data/06087.txt',
 'data/10381.txt',
 'data/05954.txt',
 'data/05940.txt',
 'data/08294.txt',
 'data/10424.txt',
 'data/06483.txt',
 'data/09767.txt',
 'data/06440.txt',
 'data/09773.txt',
 'data/08451.txt',
 'data/08109.txt',
 'data/10008.txt',
 'data/09559.txt',
 'data/06118.txt',
 'data/07204.txt',
 'data/09573.txt',
 'data/08862.txt',
 'data/10208.txt',
 'data/08255.txt',
 'data/07614.txt',
 'data/06293.txt',
 'data/05942.txt',
 'data/06052.txt',
 'data/06091.txt',
 'data/10195.txt',
 'data/05943.txt',
 'data/10369.txt',
 'data/06325.txt',
 'data/07007.txt',
 'data/07211.txt',
 'data/09562.txt',
 'data/07017.txt',
 'data/07765.txt',
 'data/09990.txt',
 'data/05984.txt',
 'data/05562.txt',
 'data/10184.txt',
 'data/07189.txt',
 'data/07823.txt',
 'data/09629.txt',
 'data/07162.txt',
 'data/05991.txt',
 'data/07002.txt',
 'data/10218.txt',
 'data/06334

In [34]:
local_file_path="eng-to-cypher-trng.jsonl"

In [54]:
from google.cloud import storage
from timeit import default_timer as timer

bucket_name = project_id + '-genai'
client = storage.Client(credentials=credentials)
try:
    bucket = client.get_bucket(bucket_name)
except:
    bucket = client.bucket(bucket_name)
    bucket.storage_class = 'STANDARD'
    bucket = client.create_bucket(bucket)

upload_name = f"eng-to-cypher-trng-{timer()}.jsonl" #this ensures vertexai reloads the file
filename = 'eng-to-cypher-trng.jsonl'
blob = bucket.blob(upload_name)
blob.upload_from_filename(filename)

In [55]:
# Assuming 'local_file_path' is the path to your local training data file in JSONL format
training_data = 'gs://' + bucket_name + '/' + upload_name

# Initialize Vertex AI with the project and location
vertexai.init(project=project_id,credentials=credentials)

# Load a pre-trained text generation model
model = TextGenerationModel.from_pretrained("text-bison@001")

# Fine-tune the model with the local JSONL file


In [56]:
tuned_model_names = model.list_tuned_model_names()
print(tuned_model_names)

['projects/209280395857/locations/us-central1/models/1718472902539476992']


In [57]:
tuned_model_name="projects/209280395857/locations/us-central1/models/1718472902539476992"

In [62]:
def english_to_cypher_text_bison(prompt, tuned_model_name = ''):
    try:
        res = run_text_model(project_id, "text-bison@001", 0.1, 1024, 0.95, 40, prompt, location, tuned_model_name)
        return res
    except Exception as e:
        print(e)

In [58]:
samples = """
Question: How many expert java developers attend more than one universities?
Answer: MATCH (p:Person)-[:HAS_SKILL]->(s:Skill), (p)-[:HAS_EDUCATION]->(e1:Education), (p)-[:HAS_EDUCATION]->(e2:Education) WHERE toLower(s.name) CONTAINS 'java' AND toLower(s.level) CONTAINS 'expert' AND e1.university <> e2.university RETURN COUNT(DISTINCT p)

Question: Where do most candidates get educated?
Answer: MATCH (p:Person)-[:HAS_EDUCATION]->(e:Education) RETURN e.university, count(e.university) as alumni ORDER BY alumni DESC LIMIT 1

Question: How many people have worked as a Data Scientist in San Francisco?
Answer: MATCH (p:Person)-[:HAS_POSITION]->(pos:Position) WHERE toLower(pos.title) CONTAINS 'data scientist' AND toLower(pos.location) CONTAINS 'san francisco' RETURN COUNT(p)
"""

In [59]:
prompt = """You are an expert Neo4j Cypher translator who understands the question in english and convert to Cypher strictly based on the Neo4j Schema provided and following the instructions below:
1. Generate Cypher query compatible ONLY for Neo4j Version 5
2. Do not use EXISTS, SIZE keywords in the cypher. Use alias when using the WITH keyword
3. Use only Nodes and relationships mentioned in the schema
4. Always enclose the Cypher output inside 3 backticks
5. Always do a case-insensitive and fuzzy search for any properties related search. Eg: to search for a Company name use `toLower(c.name) contains 'neo4j'`
6. Candidate node is synonymous to Person
7. Always use aliases to refer the node in the query
8. Cypher is NOT SQL. So, do not mix and match the syntaxes
Schema:
(:Person {label: 'Person', id: string, role: string, description: string})-[:HAS_POSITION {}]->(:Position {label: 'Position', id: string, title: string, location: string, startDate: string, endDate: string, url: string})
(:Position {label: 'Position', id: string, title: string, location: string, startDate: string, endDate: string, url: string})-[:AT_COMPANY {}]->(:Company {label:'Company', id: string, name: string})
(:Person {label: 'Person',id: string, role: string, description: string})-[:HAS_SKILL {}]->(:Skill {label:'Skill', id: string,name: string,level: string})
(:Person {label: 'Person',id: string, role: string, description: string})-[:HAS_EDUCATION {}]->(:Education {label:'Education', id: string, degree: string, university: string, graduationDate: string, score: string, url: string})
Samples:
$samples
Question: $question
Answer:
"""

In [60]:
tuned_model_name

'projects/209280395857/locations/us-central1/models/1718472902539476992'

In [63]:
from string import Template
que = 'How many are knowledgable on all of - java, python, javascript, security?'
_prompt = Template(prompt).substitute(samples=samples, question=que)

cypher = english_to_cypher_text_bison(_prompt, tuned_model_name) #for text-bison use: english_to_cypher_text_bison(_prompt, entity_extraction_tuned_model)
if 'Answer:\n ' in cypher:
    cypher = cypher.split('Answer:\n ')[1]
cypher = cypher.replace('\n', ' ')
cypher

"MATCH (p:Person)-[:HAS_SKILL]->(s1:Skill), (p)-[:HAS_SKILL]->(s2:Skill), (p)-[:HAS_SKILL]->(s3:Skill), (p)-[:HAS_SKILL]->(s4:Skill) WHERE toLower(s1.name) CONTAINS 'java' AND toLower(s2.name) CONTAINS 'python' AND toLower(s3.name) CONTAINS 'javascript' AND toLower(s4.name) CONTAINS 'security' RETURN COUNT(DISTINCT p)"

In [60]:
tuned_model_name

'projects/209280395857/locations/us-central1/models/1718472902539476992'

In [61]:
model = model.get_tuned_model(tuned_model_name)

In [62]:
model

In [63]:
from google.cloud import aiplatform as aip
my_model = aip.Model("projects/209280395857/locations/us-central1/models/1718472902539476992")

In [64]:
my_model

resource name: projects/209280395857/locations/us-central1/models/1718472902539476992

In [65]:
model=model.get_tuned_model("projects/209280395857/locations/us-central1/models/1718472902539476992")

In [66]:
model

In [64]:
tuned_model_names = model.list_tuned_model_names()
print(tuned_model_names)

['projects/209280395857/locations/us-central1/models/1718472902539476992']


In [31]:
%pip install langchain
from langchain.llms import VertexAI


llm = VertexAI(model_name="text-bison@001", 
               tuned_model_name="projects/209280395857/locations/us-central1/models/1718472902539476992",
               max_output_tokens=1024)

Note: you may need to restart the kernel to use updated packages.


C:\Users\Pratik's Predator\AppData\Roaming\Python\Python310\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.vertexai.VertexAI` was deprecated in langchain-community 0.0.12 and will be removed in 0.2.0. An updated version of the class exists in the langchain-google-vertexai package and should be used instead. To use it run `pip install -U langchain-google-vertexai` and import as `from langchain_google_vertexai import VertexAI`.
  warn_deprecated(


In [32]:
llm

VertexAI(model_name='text-bison@001', client=<vertexai.language_models.TextGenerationModel object at 0x0000023F0CF1FCD0>, client_preview=<vertexai.preview.language_models._PreviewTextGenerationModel object at 0x0000023F0D176E90>, max_output_tokens=1024, tuned_model_name='projects/209280395857/locations/us-central1/models/1718472902539476992')

In [48]:
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph
from langchain.llms import VertexAI
from langchain.prompts.prompt import PromptTemplate

CYPHER_GENERATION_TEMPLATE = """You are an expert Neo4j Cypher translator who understands the question in english and convert to Cypher strictly based on the Neo4j Schema provided and following the instructions below:
1. Generate Cypher query compatible ONLY for Neo4j Version 5
2. Do not use EXISTS, SIZE keywords in the cypher. Use alias when using the WITH keyword
3. Use only Nodes and relationships mentioned in the schema
4. Always enclose the Cypher output inside 3 backticks
5. Always do a case-insensitive and fuzzy search for any properties related search. Eg: to search for a Company name use `toLower(c.name) contains 'neo4j'`
6. Candidate node is synonymous to Person
7. Always use aliases to refer the node in the query
8. Cypher is NOT SQL. So, do not mix and match the syntaxes
Schema:
{schema}
Samples:
Question: How many expert java developers attend more than one universities?
Answer: MATCH (p:Person)-[:HAS_SKILL]->(s:Skill), (p)-[:HAS_EDUCATION]->(e1:Education), (p)-[:HAS_EDUCATION]->(e2:Education) WHERE toLower(s.name) CONTAINS 'java' AND toLower(s.level) CONTAINS 'expert' AND e1.university <> e2.university RETURN COUNT(DISTINCT p)
Question: Where do most candidates get educated?
Answer: MATCH (p:Person)-[:HAS_EDUCATION]->(e:Education) RETURN e.university, count(e.university) as alumni ORDER BY alumni DESC LIMIT 1
Question: How many people have worked as a Data Scientist in San Francisco?
Answer: MATCH (p:Person)-[:HAS_POSITION]->(pos:Position) WHERE toLower(pos.title) CONTAINS 'data scientist' AND toLower(pos.location) CONTAINS 'san francisco' RETURN COUNT(p)
Question: {question}
Answer:
"""

CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE
)


graph = Neo4jGraph(
    url="neo4j+s://fe656664.databases.neo4j.io", 
    username="neo4j", 
    password="admin123"
)
chain = GraphCypherQAChain.from_llm(
    llm = llm,
    graph=graph, verbose=False,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
    return_intermediate_steps=False
)

ValueError: Could not connect to Neo4j database. Please ensure that the username and password are correct

In [43]:
r = chain("""Which people have held a position in London with a start date in 2019""")

In [44]:
#print(f"Intermediate steps: {r['intermediate_steps']}")
print(f"Final answer: {r['result']}")

Final answer:  I'm sorry, but I cannot answer this question as the provided information does not contain any details about people who have held a position in London with a start date in 2019.


In [45]:

import gradio as gr
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key = "chat_history", return_messages = True)
llm = VertexAI(model_name='code-bison',
            max_output_tokens=2048,
            temperature=0,
            top_p=0.95,
            top_k=0.40)
agent_chain = chain
def chat_response(input_text):
    response = agent_chain.run(input_text)
    return response

interface = gr.Interface(fn = chat_response, inputs = "text", outputs = "text", 
                         description = "Talent Finder Chatbot")

interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7862
Running on public URL: https://536137caffe0648bd8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "C:\Users\Pratik's Predator\AppData\Roaming\Python\Python310\site-packages\google\api_core\grpc_helpers.py", line 76, in error_remapped_callable
    return callable_(*args, **kwargs)
  File "C:\Users\Pratik's Predator\AppData\Roaming\Python\Python310\site-packages\grpc\_channel.py", line 1160, in __call__
    return _end_unary_response_blocking(state, call, False, None)
  File "C:\Users\Pratik's Predator\AppData\Roaming\Python\Python310\site-packages\grpc\_channel.py", line 1003, in _end_unary_response_blocking
    raise _InactiveRpcError(state)  # pytype: disable=not-instantiable
grpc._channel._InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.INVALID_ARGUMENT
	details = "Request contains an invalid argument."
	debug_error_string = "UNKNOWN:Error received from peer ipv6:%5B2607:f8b0:4006:80d::200a%5D:443 {grpc_message:"Request contains an invalid argument.", grpc_status:3, created_time:"2024-04-25T00:12:03.

[0.030059814,
 0.013595581,
 -0.03265381,
 -0.087524414,
 0.0025405884,
 -0.013252258,
 -0.03869629,
 0.02960205,
 0.032104492,
 0.02583313,
 -0.0129776,
 -0.01537323,
 0.01360321,
 -0.024993896,
 0.018737793,
 -0.014518738,
 0.035064697,
 0.012290955,
 0.041107178,
 -0.016098022,
 0.007259369,
 0.0073394775,
 -0.051330566,
 0.016479492,
 0.024765015,
 0.019012451,
 -0.06890869,
 -0.004131317,
 0.03744507,
 0.036621094,
 0.013793945,
 -0.013809204,
 0.03845215,
 -0.0056877136,
 -0.016036987,
 -0.0037384033,
 0.037750244,
 -0.030059814,
 -0.020401001,
 0.03552246,
 0.036102295,
 -0.024215698,
 -0.042388916,
 -0.0003671646,
 -0.052490234,
 -0.043548584,
 0.07495117,
 0.007820129,
 0.026794434,
 -0.03201294,
 -0.038879395,
 -0.030853271,
 -0.029891968,
 -0.00724411,
 -0.03010559,
 0.010726929,
 -0.024871826,
 -0.017089844,
 0.015701294,
 -0.037231445,
 -0.017425537,
 -0.010650635,
 0.038330078,
 -0.0014238358,
 -0.03036499,
 -0.0061950684,
 0.002445221,
 0.013916016,
 0.054534912,
 -0.021

In [3]:
len(em)

1024

In [59]:
em2 = [-0.02528921, 0.004882717, -0.027767671, -0.034193862, -0.030275814, 0.040961396, -0.0077470466, 0.038735233, 0.0012206793, 0.0019423271, 0.035054643, -0.062510654, -0.0038846543, 0.0026806712, -0.0019627335, 0.0017466103, -0.028346473, 0.030632, -0.04223773, -0.03175992, 0.046660375, -0.0240574, -0.050103504, 0.03639034, -0.00073741644, 0.025645396, -0.0032186604, 0.006737853, 0.018699767, 0.006329723, 0.024042558, 0.017245341, -0.008785923, 0.00587707, 0.0013050879, -0.03947729, -0.008095812, -0.016117418, 0.024101922, -0.01681495, 0.006552339, -0.00038633202, 0.015969008, 0.03208643, 0.050608102, 0.023582485, -0.014915291, -0.010544592, -0.011672514, -0.008414895, 0.0033967535, -0.023864465, -0.014128714, 0.045621496, 0.008771081, -0.018744292, 0.0068528713, 0.043662474, -0.014714937, 0.023493439, 3.9892666e-06, 0.04060521, -0.02172735, -0.014752039, 0.014781721, 0.021089183, -0.0070532258, 0.00424084, 0.016473604, 0.0058844904, -0.022974001, 0.0004939299, -0.015405047, 0.030275814, 0.022202265, 0.0083703725, 0.021578938, -0.0144700585, -0.0038772337, -0.02365669, 0.029904788, 0.013052735, -0.013750265, -0.025215004, 0.018699767, -0.012043541, -0.018017078, -0.093973756, 0.01736407, -0.0027381803, 0.018402945, -0.0011789388, 0.00027780657, -0.02187576, 0.03460941, 0.01080431, 0.020050306, 0.017690575, -0.0011334879, -0.024294857, 0.008563306, -0.02024324, -0.007127431, -0.028272267, 0.02898464, -0.04238614, -0.014811403, -0.011961915, 0.02229131, 0.033036258, -0.004671232, 0.015657345, 0.02460652, 0.028109016, -0.0060625835, -0.017245341, -0.0038178694, -0.04384057, -0.008110654, -0.026832683, -0.020391652, 0.026016423, 0.00382529, 0.025600873, -0.0028773155, -0.040812984, 0.013186305, -0.0044226437, 0.019085636, 0.017453117, -0.032887846, 0.021653144, 0.010611376, -0.0013097258, 0.025586031, 0.03935856, 0.01708209, 0.0131640425, 0.00043502933, 0.004140663, -0.008763661, -0.00083713, 0.008229382, -0.015538617, 0.0278864, 0.025868012, -0.01763121, 0.03101787, -0.0062814895, 0.033006575, -0.030290656, 0.022335833, -0.033154987, -0.00588078, 0.053249814, 0.015627664, 0.02610547, 0.05811769, -0.03719176, 0.0017438276, -0.0048752967, -0.0017540308, -0.015761232, 0.006396508, 0.05057842, -0.009758014, 0.012681708, -0.0043929615, -0.040367752, 0.02665459, -0.021044658, -0.0134015, 0.026046105, -0.009691229, -0.018625563, -0.028940117, 0.011561206, 0.0106930025, -0.031403735, -0.011249543, -0.04401866, 0.020510381, -0.013601855, 0.00019594871, 0.01408419, 0.00861525, 0.023434075, 0.014388433, 0.020169035, -0.03784477, 0.0045042695, -0.011791242, 0.018328741, -0.010559432, 0.03606384, 0.0071459827, 0.015345682, 0.04541372, 0.032739434, -0.06672552, 0.009654126, 0.006114527, 0.0053242394, 0.014343909, -0.036568437, 0.007680262, -0.009988051, -0.014989496, -0.0065300777, 0.007806411, -0.014996917, 0.011665094, 0.041911226, 0.015909644, -0.032472294, -0.0069382074, 0.027559895, -0.009691229, 0.027248234, -0.027841877, 0.052121893, 0.031136598, 0.011679934, 0.0035859772, -0.023537962, -0.01681495, -0.009127268, 0.02270686, -0.03549988, 0.022098377, -0.037221443, 0.0018263811, 0.004897558, -0.00998063, 0.02775283, 0.017942872, 0.023567643, 0.024933023, -0.057197545, 0.03416418, 0.0151601685, -0.00984706, -0.021118864, -0.040456798, 0.045443404, 0.0028661846, 0.011360851, 0.052863948, 0.026342927, 0.019026272, 0.0089862775, -0.028153539, 0.014536844, 0.02830195, 0.03101787, 0.02855425, -0.023879306, -0.005780603, 0.02309273, 0.03692462, -0.014848506, -0.012718811, 0.00014365707, 0.011665094, -0.020970454, 0.013542491, 0.0069716, 0.018106125, 0.017542163, -0.013223407, 0.039121103, -0.024710407, 0.038646188, 0.017260183, 0.015375365, -0.030053198, 0.011516683, 0.026016423, 0.0070458055, -0.02595706, 0.015145328, -0.0064299004, 0.017809303, 0.034698457, -0.012948847, 0.010685582, 0.04541372, -0.032294203, 0.04648228, -0.014314227, -0.01941214, -0.017601527, 0.010136462, -0.012370045, 0.014477479, 0.0042445506, 0.014967235, -0.004938371, 0.021400845, 0.008808184, -0.0017262037, 0.023033366, -0.017438276, -0.038379047, -0.0012893194, 0.015835438, -0.04769925, -0.008459419, -0.029860264, -0.0104407035, 0.018358422, -0.01189513, -0.03763699, -0.0077173645, 0.015479253, 0.03270975, 0.03549988, -0.01490045, -0.03184897, -0.015011759, 0.004337307, 0.00034551902, -0.04048648, -0.0021705087, 0.029771218, 0.011271805, -0.02310757, 0.06577569, -0.017838985, -0.02309273, -0.049836364, -0.013371818, 0.013572173, -0.036657482, 0.0058288365, -0.019590233, -0.033184666, 0.014343909, -0.03131469, 0.015316, 0.009520557, 0.018299058, -0.049717635, 0.0064299004, 0.0025248397, -0.034461, 0.045888636, -0.02612031, -0.033273716, -0.016458765, 0.0007499386, -0.05482297, 0.058177054, -0.025660237, -0.033837676, -0.0077693085, 0.02432454, 0.013935779, -0.0031203383, -0.011583467, 0.037221443, -0.046660375, 0.0020592005, -0.005086782, 0.016651697, 0.027411485, 0.027559895, -0.0041888966, 0.00984706, -0.025675079, 0.01613226, 0.019560551, 0.024443269, 0.060373537, -0.058978476, -0.0038846543, -0.0055728275, 0.035024963, -0.020940771, 0.013705743, -0.024443269, -0.017942872, -0.042356458, 0.0003371709, 0.018714609, 0.013557332, 0.011917392, -0.042297095, 0.00075782294, -0.0026324375, -0.01749764, 0.0050088665, -0.022602974, -0.010737525, 0.013431182, -0.015657345, -0.0040108035, 0.018477151, 0.049747318, -0.03909142, -0.0057063974, 0.019590233, -0.05342791, -0.020822043, -0.016147101, -0.022617815, -0.01297853, -0.0042631016, 0.013082417, 0.01626583, -0.020391652, -0.0121103255, 4.568272e-05, -0.03035002, -0.005888201, 0.009535397, 0.028925275, 0.023478597, -0.0018078297, 0.023003682, 0.006463293, 0.024027718, 0.014076769, 0.009936106, -0.04021934, -0.0021538124, 0.023716055, 0.016725903, 0.05963148, 0.028242586, 0.012503615, -0.032472294, 0.023315346, -0.015924485, 0.023493439, -0.016785268, 0.006478134, 0.006548629, 0.00683432, 0.0062963306, -0.051557932, 0.011679934, 0.00956508, -0.0123626245, -0.07432415, 0.012511035, -0.016666539, 0.013631538, -0.024369063, 0.03455005, 0.029133052, 0.022098377, -0.0023448914, 0.020584585, -0.02255845, -0.02052522, -0.012815277, 0.0352921, 0.006463293, 0.0052611646, -0.020272924, 0.012065803, 0.0040441956, -0.0107672075, -0.007465066, -0.00090994406, -0.022068694, -0.030721048, 0.012384886, 0.061264, 0.026283562, -0.015835438, -0.005123885, 0.007097749, 0.024369063, 0.02898464, 0.04137695, -0.0100399945, 0.006478134, -0.027856717, 0.004563634, 0.024992388, -0.039625697, 9.304665e-05, 0.066072516, 0.027055299, 0.0137428455, -0.015672186, -0.003309562, 0.038557142, -0.0072016367, -0.018120965, -0.012266157, -0.028524566, -0.044374846, -0.010856254, -0.014202919, 0.018922385, -0.027366962, -0.030246133, -0.008311008, 0.02937051, 0.03499528, 0.0123626245, 0.013023052, -0.007672841, 0.02377542, -0.051617295, -0.008021607, -0.023834784, -0.010084517, 0.023077888, 0.0008705224, -0.0018848178, -0.018017078, -0.0040293545, -0.0010314555, -0.01052975, -0.0054763607, 0.013579593, 0.006084845, 0.01791319, 0.025096275, -0.010685582, 0.031255327, -0.0064744237, -0.0083332695, -0.025333732, -0.0014154685, -0.021267276, 4.4697175e-05, -0.00075272133, -0.016993042, -0.010344237, -0.05004414, 0.0043669892, 0.00188018, 0.0047751195, 0.019857373, -0.018744292, -0.012637184, 0.018091284, -0.014395853, -0.00027827037, 0.007984504, 0.0041851862, -0.010218088, -0.012867222, 0.018684927, -0.0120732235, -0.014039667, 0.019872213, 0.007821253, 0.017408593, 0.01941214, -0.02134148, -0.009364725, -0.0076357387, 0.041317582, -0.0043669892, -0.023686372, -0.02021356, 0.020673633, 0.020139353, -0.012140008, 0.017586686, -0.015241795, 0.059809577, -0.047402427, -0.0009155095, 0.021831237, -0.027233392, -0.05387314, -0.0046304185, -0.02502207, 0.027975446, -0.001483181, -0.014343909, -0.024933023, 0.00021902195, 0.0057472102, -0.004400382, 0.005498622, 0.002009112, -0.0055839582, 0.041198853, 0.0064002182, 0.009965789, -0.004912399, -0.031344373, 0.023463756, -0.0025081434, 0.021163387, -0.00424084, -0.00065579044, 0.011375692, 0.011397954, 0.008904652, 0.03668716, -0.025333732, 0.00059317966, 0.059037838, -0.03541083, 0.0045042695, -0.026446814, 0.034253225, -0.029058846, -0.030038357, 0.0042037377, 0.01736407, -0.05767246, -0.027426327, 0.015256636, 0.012191952, 0.009053063, 0.009668968, -0.01504144, -0.008177438, -0.008296167, 0.0072795525, 0.015212113, -0.0151601685, -0.010774628, -0.039061736, 0.010596535, 0.0051609874, 0.017720256, -0.015464411, -0.005784313, -0.013468285, -0.0011613149, 0.017735098, -0.005402155, -0.018135807, -0.00061590504, 0.014076769, 0.019115318, -0.01693368, 0.022766225, 0.032056745, -0.0017911335, 0.004663811, -0.0081997, 0.06464777, -0.015865121, 0.020970454, 0.033807993, 0.034668777, 0.037785403, -0.022602974, -0.022617815, -0.009936106, 0.025749283, 0.013208566, -0.01721566, -0.011835766, -0.015894802, -0.016429082, -0.0040441956, -0.0071867956, 2.5768994e-05, 0.0009989906, 0.005235193, -0.032116108, 0.024918182, 0.03389704, -0.02009483, 0.028598772, -0.040516164, 0.05452615, 0.017052406, -0.0028457781, -0.021237593, 0.032027062, -0.0141064515, -0.0075429818, -0.016058054, -0.003580412, -0.008860128, -0.006262938, -0.0022910926, 0.0029719274, -0.0034616832, 0.0278864, 0.036479387, 0.016443923, -0.042979784, -0.022988841, 0.053813778, -0.0013774383, -0.029978992, -0.002445069, 0.022513926, -0.020896249, 0.016147101, -0.010500068, -0.03143342, -0.041881543, -0.0006108034, -0.010611376, -0.010425863, -0.026431974, -0.008444577, 0.009513136, -0.033154987, 0.0072572907, -0.010819151, -0.030112563, 0.010915618, -0.024769772, 0.018714609, -0.0009674533, -0.01366864, -0.025452461, -0.011642831, 0.010893357, -0.0037714909, 0.014982076, -0.0074131223, -0.0028958668, -0.0009674533, 0.008956595, -0.029489238, -0.018239694, -0.06512269, 0.02147505, 0.0014126858, -0.0028327922, -0.0014126858, -0.005494912, -0.010908198, -0.0014089756, 0.012280999, 0.0027975447, -0.009201473, 0.015954167, 0.019887054, 0.018269377, 0.023641849, 0.04105044, 0.017304705, -0.013193725, 0.010507489, 0.03048359, -0.016117418, -0.004700914, -0.007806411, 0.018773973, 0.012548137, -0.013193725, 0.011442477, -0.025586031, 0.008214541, -0.016295511, 0.012177111, 0.024665885, 0.012243896, 0.008103233, 0.013505388, -0.008964016, -0.049747318, -0.005457809, -0.020985294, 0.02175703, 0.010418442, 0.017809303, 0.04781798, -0.027975446, 0.020287763, 0.009661547, 0.0037547946, 0.014395853, 0.02352312, -0.005305688, 0.022588132, 0.017868668, -0.0076060565, -0.017200818, -0.006986441, -0.0030869457, 0.0035711362, -0.010218088, -0.06773472, 0.021638302, -0.024784613, -0.012013859, 0.011219861, -0.020970454, -0.027233392, 0.0023133543, 0.026535861, -0.014306807, 0.033333078, 0.00071329967, 0.007873196, -0.0045339516, 0.004456036, 0.0052982676, -0.0012642749, 0.03633098, -0.0077173645, 0.00015977357, -0.0019627335, -0.0073500476, 0.008236802, -0.026491338, -0.009728332, -0.03541083, -0.012295839, -0.003730678, 0.00069382077, -0.01285238, 0.0036416315, 0.0038735233, -0.008860128, -0.007999345, 0.0155237755, 0.0028717502, -0.04099108, 0.0062555177, -0.016844632, -0.045888636, -0.008793343, -0.026550703, -0.0007462283, 0.016740745, 0.0017827854, 0.016592333, 0.012659446, -0.018521674, -0.0063371435, -0.022899795, -0.030988187, -0.026758477, -0.016429082, 0.023686372, -0.007977083, 0.0259719, -0.030869458, -0.011271805, 0.028257428, -0.008756241, -5.863968e-05, 0.017408593, 0.008058709, 0.014425535, -0.00026389305, -0.025942218, -0.0007068067, 0.019649597, -0.0035377438, -0.013342136, -0.04021934, 0.02978606, 0.013653799, 0.015597981, -0.023285663, 0.041228537, 0.022810748, -0.019456664, -0.023211459, -0.03208643, -0.013186305, -0.00478625, 0.024131605, -0.015360524, -0.021786714, -0.0015017324, 0.032205157, 0.011642831, 0.002858764, 0.0008426954, -0.0019163552, -0.0037677805, -0.018625563, 0.005906752, -0.012154849, 0.02735212, 0.000536598, 0.01982769, 0.013987723, -0.03360022, -0.010292293, 0.033451807, -0.0063223024, -0.0118209245, -0.049242724, 0.0248885, -0.009201473, 0.004055327, -0.009884163, 0.044612303, 0.0037343882, -0.028954959, 0.029162733, 0.0010129041, 0.016250988, 0.0047899606, -0.017868668, 0.021222752, -0.004893848, -0.0090604825, -0.03238325, 0.02775283, 0.00969865, 0.012258736, 0.020154195, -0.008377793, -0.0042037377, -0.053546637, 0.00020661573, 0.022276469, -0.0003378666, -0.03707303, 0.022276469, 0.019976102, -0.033422124, -0.01258524, 0.00464897, -0.031077234, 0.00028406765, 0.027084982, 0.003676879, 0.022602974, 0.015627664, 0.0029274041, -0.002569363, -0.031255327, 0.011360851, 0.0014043377, 0.009053063, 0.00738344, -0.017675733, -0.017467957, 0.04591832, -0.0003343882, 0.011583467, 0.00017762925, -0.018239694, -0.030780412, 0.0006794435, 0.026313245, -0.017423434, 0.001805047, -0.0049791844, 0.010099359, 0.03006804, 0.0039996724, 0.01079689, 0.0045970264, 0.02324114, -0.0026992224, 0.014024826, -0.014002564, 0.0016074751, -0.022320993, -0.01763121, -0.061323367, -0.0051201745, 0.01736407, 0.00409985, 0.040456798, -0.03360022, -0.009691229, -0.008014186, 0.008036448, -0.00026969035, 0.023478597, -0.021831237, -0.00031119902, -0.0034524074, -0.020822043, 0.0019145, -0.0043966714, 0.021460209, 0.017853826, -0.0010657754, -0.0027548766, -0.013045315, 0.00478254, 0.0023986904, -0.019501187, -0.017660892, 0.001066703, 0.048886538, -0.020822043, 0.007991925, -0.0036397763, 0.04321724, 0.0180616, -0.03404545, -0.027055299, -0.012644605, -0.0071014594, 0.028287109, -0.015167589, 0.024220651, -0.047254015, 0.007709944, -0.014915291, -0.0050088665, -0.0032001091, -0.004337307, 0.018388106, -0.007910298, 0.02432454, -0.022647496, 0.03636066, -0.015954167, -0.00021948574, -0.00096467056, 0.018210012, -0.01954571, -0.040456798, -0.0007207202, 0.0038104488, 0.023255982, 0.01244425, -0.00012452598, -0.022899795, -0.019634755, 0.0010119765, 0.01285238, -0.0060885553, 0.004882717, 0.026001582, -0.003376347, 0.008637512, -0.0045747645, 0.006340854, -0.024250334, -0.012273578, -0.004274233, -0.009884163, -0.0065263673, -0.02542278, -0.0103293955, -0.043781202, 0.0060329014, -0.016518129, 0.032442614, -0.0144329555, 0.008548466, 0.020465856, -0.012540718, 0.004318756, -0.017735098, 0.013958041, -0.011242122, -0.012258736, -0.04401866, -0.039299194, 0.005713818, 0.0007388078, 0.00036499795, 0.0036045287, -0.010218088, 0.011865448, 0.02623904, 0.0030925113, 0.0016557086, -0.035767015, 0.010284873, 0.007479907, -0.0031611512, 0.0117467195, -0.045562133, -0.018284218, -0.012318101, 0.008682035, -0.009750593, 0.008080971, 0.022528768, -0.004608157, -0.008741399, 0.012125167, -0.006481844, 0.0066636475, -0.026728796, -0.0036527622, -0.011813504, -0.010923039, -0.01887786, 0.0035748465, 0.016028373, -0.0047788294, -0.014232601, 0.014024826, -0.02021356, 0.0077322056, -0.027827036, -0.016993042, 0.012377465, -0.0048048017, 0.0040033828, 0.004812222, -0.0026157412, 0.054971382, -0.0014349475, -0.0080438685, -0.027129505, -0.011702196, -0.03680589, 0.014559105, -0.007999345, 0.007064357, -0.0038327104, 0.0039328877, 0.014855927, -0.0031129178, -0.01954571, 0.024502633, -0.0074353837, -0.003047988, 0.0128004365, 0.032561343, -0.034906235, -0.0031426, 0.01490787, 0.0015100804, 0.010611376, -0.0028513435, 0.004827063, 0.01668138, -0.020584585, -0.04882717, -0.015701868, -0.006901105, -0.009357304, 0.03416418, 0.018417787, -0.015419888, 0.018521674, 0.0058993315, -0.017883508, 0.024502633, 0.0022743964, -0.01573155, -0.0043113353, 0.007465066, 0.042593915, -0.026417132, -0.019708961, 0.0029923338, -0.016058054, 0.031136598, 0.0019497477, 0.0002794298, -0.0240574, 0.0039440184, 0.019382458, 0.013305034, 0.0103293955, 0.023122411, 0.012206793, 0.00024742872, 0.02542278, 0.006934497, -0.018670086, 0.019308252, -0.0025804937, -0.008800764, 0.021267276, 0.019278571, -0.0029552311, -0.02720371, -0.0106930025, -0.016340036, 0.0072721317, -0.0016501432, 0.01996126, -0.035529558, 0.040812984, 0.0036954302, -0.026417132, 0.025689919, -0.0006047742, -0.040456798, 0.018269377, -0.013891256, 0.028272267, 0.00044963852, 0.012013859, 0.003580412, -0.0034301458, -0.0071979263, 0.0048604556, -0.027381804, 0.010833993, 0.02295916, -0.0028142408, 0.013379239, 0.02528921, -0.0008246078, -0.0063816668, -0.0017475379, 0.0043484382, 0.009253417, 0.00044871095, 0.025541509, -0.046037048, 0.012934006, -0.013186305, 0.009053063, 0.02267718, 0.0051053334, -0.0043818303, -0.0061442093, -0.019159842, -0.029875105, -0.020584585, -0.0054726503, 0.013141781, -0.022024171, 0.011627991, 0.0017929886, 4.101589e-05, -0.01898175, 0.013356977, 0.011954495, -0.014818824, 0.0067044604, -0.0073760194, 0.005784313, -0.008348111, 0.0357967, -0.005825126, 0.011004665, 0.0069716, -0.0019070795, -0.013208566, -0.0248885, -0.007902878, -0.00546894, -0.01887786, 0.008956595, 0.006496685, 0.003977411, 0.0003698677, 0.008422316, 0.01571671, -0.010433284, -0.005717528, -0.05369505, 0.0050942027, 0.002172364, -0.0017753649, -0.00792514, -0.015969008, -0.017319547, -0.023686372, 0.03938824, 0.0057991543, -0.009550239, -0.00022621061, 0.025259528, 0.0058919108, 0.026031265, -0.0043261764, -0.036182567, -0.0031889782, -0.021074342, 0.000635384, 0.018239694, 0.025793808, -0.003649052, -0.00560622, -0.010017733, 0.02009483, -0.006941918, -0.002066621, -0.0019924156, -0.024621362, 0.02324114, 0.007565243, -0.0036360659, 0.035054643, 0.006615414, -0.009602183, 0.0038772337, 0.015642505, 0.01573155, -0.022335833, 0.0026732506, -0.01926373, -0.009728332, 0.019560551, -0.03297689, 0.017928032, 0.022365516, -0.042326774, 0.015790915, -0.0017002319, -0.0042074476, -0.009631865, -0.0016213886, -0.0040108035, -0.026046105, -0.0147965625, -0.013045315, 0.020792361, -0.014811403, 0.010099359, -0.008808184, -0.0006789797, -0.004170345, 0.008971436, 0.014944973, 0.015820596, 0.011724458, 0.02557119, -0.007999345, -0.039032057, 0.03116628, 0.0021315508, 0.0015444005, 0.0017002319, -0.0017605238, 0.006300041, -0.007954822, 0.007149693, -0.019145, 0.00765058, -0.050875243, 0.007001282, 0.0065078163, 0.007424253, -0.0027975447, 0.026283562, -0.0010982404, -0.020851726, -0.00018933979, -0.008860128, 0.0010833993, -0.017972555, -0.021386005, -0.028049652, 0.017883508, -0.008837867, -0.007487328, 0.021400845, -0.004218579, -0.00023792115, -0.0123997275, -0.019219207, -0.014284545, 0.018373264, -0.035440512, -0.0025471014, 0.013483127, -0.0039848313, -0.0010351657, -0.0052203517, -0.009275679, 0.025304051, 0.013527649, 0.026164833, -0.003213095, 0.0046563908, 0.011924813, -0.004118401, 0.0040145135, 0.008355531, 0.0022725412, -0.023211459, -0.0028365024, 0.019382458, -0.029563442, -0.01175414, -0.0029478106, -0.007479907, 0.035767015, -0.0035989631, 0.008132915, 0.017557004, -0.0033986086, 0.0045970264, 0.008978857, 0.03594511, 0.01898175, -0.002717774, -0.026328085, 0.03689494, 0.021964807, 0.013594435, -0.0016102578, -0.01967928, -0.002856909, 0.005168408, 0.007283263, -0.008236802, -0.016607175, -0.0007657073, -0.0048233527, 0.024843978, 0.007873196, -0.0047454373, 0.012221634, -0.012392307, 0.010878515, -0.003840131, 0.008778502, -0.021504734, 0.0067081708, -0.0352921, 0.009943527, 0.008533624, 0.030780412, -0.0062147044, 0.02925178, -0.0068009277, 0.002951521, 0.01079689, 0.00943893, 0.012080643, -0.02747085, 0.0065931524, 0.024977546, 0.018744292, 0.015093384, -0.00765058, -0.0064373207, -0.0104407035, -0.028346473, -0.0057212384, -0.0031648616, -0.00013554085, 0.003060974, -0.023953512, -0.003747374, 0.024012877, -0.00820712, -0.0026194516, 0.02270686, 0.012815277, 0.035529558, 0.021489892, -0.016622016, 0.008756241, 0.01107145, 0.0030201608, -0.021252435, 0.029177574, 0.002363443, -0.0070903287, -0.006433611, 0.011613149, -0.027693465, 0.012934006, -0.026194517, -0.0048196428, 0.0075763743, -0.0024951575, -0.027381804, 0.0058065746, 0.012325522, 0.02186092, 0.008860128, 0.013520229, 0.017141454, -0.0052982676, -0.019145, -0.025615714, -0.00846684, -0.042267412, 0.012726231, -0.023285663, -0.011509262, 0.00037914337, -0.016280672, 0.026194517, -0.016206466, 0.005591379, 0.038438413, 0.032116108, 0.008051289, -0.007821253, -0.01763121, -0.0035841223, 0.0051609874, -0.006222125, -0.0046267086, 0.024443269, 0.0147594595, 0.010752367, 0.00013217842, 0.015805757, 0.035767015, 0.019619916, 0.008214541, 0.017660892, -0.0068973945, 0.022068694, 0.026758477, -0.030171927, -0.013253089, 0.01503402, -0.0073352065, 0.013052735, 0.002706643, 0.0068120584, -0.008429737, -0.041911226, -0.008541045, -0.030201608, 0.0034524074, 0.00724616, -0.020169035, 0.002311499, -0.0077841496, -0.0068009277, -0.011828345, -0.014603628, -0.026713954, 0.022217104, 0.015479253, 0.01285238, -0.041495677, -0.010195826, 0.00917179, -0.0025396808, 0.031255327, -0.0009897149, 0.020139353, -0.007068067, -0.0059920885, 0.007821253, -0.0019905607, -0.009461192, 0.047283698, 0.01585028, -0.0109824035, -0.007446515, -0.019634755, -0.03555924, 0.014447797, 0.0103664985, 0.0025118538, 0.028509725, -0.008548466, -0.009105006, 0.015212113, -0.028005129, -0.0016788978, 0.018358422, 0.014826245, -0.0068825535, -0.0058844904, 0.008926913, -0.004567344, -0.028109016, -0.0039811214, 0.0037918973, 0.013245669, -0.0037232575, -0.021994488, -0.011509262, -0.00874882, -0.0014247443, 0.0083703725, 0.0041740555, 0.0062740687, 0.010136462, 0.0070235436, -0.013312453, 0.014447797, -0.0011474014, 0.025200164, -0.008570727, -0.022395197, 0.018120965, 0.00051155366, 0.0074984585, 0.011442477, -0.0065263673, 0.01941214, -0.0106558995, -0.004085009, -0.019798009, -0.018521674, 0.00092756783, 0.026713954, 0.062688746, 0.007672841, 0.017022725, -0.0032873005, -0.0022317283, 0.0071830857, -0.010359078, 0.00998063, -0.018299058, -0.007502169, -0.0019608785, -0.013141781, -0.014507161, 0.0069716, 0.0028179511, -0.0037547946, 0.015375365, -0.0029069977, 0.039447606, 0.00573979, -0.0117096165, -0.02267718, -0.006099686, 0.0067081708, -0.0040961397, 0.0050125765, -0.016859474, -0.00090345106, 0.0040664575, -0.0038883644, 0.0034208703, 0.0021148548, 0.022528768, -0.0030090301, 0.008882389, 0.014321648, 0.014121293, -0.017794462, -0.013379239, -8.377097e-06, -0.0031611512, 0.0070049926, 0.010188405, 0.0069901515, 0.022217104, -0.014841085, 0.003595253, -0.006492975, -0.01052975, 0.0057917335, 0.015011759, 0.0069122356, 0.0071422723, 0.015687028, 0.012770754, -0.016384559, -0.023048205, 0.0020740416, -0.008377793, 0.0073871505, -0.013616696, 0.009550239, -0.021949966, -0.0061256583, 0.0018022644, 0.003925467, 0.0026454234, 0.035648286, 0.017096931, 0.019931577, 0.0015230664, 0.0020944483, -0.01339408, -0.0022892375, 0.018848179, 0.011041768, -0.0018273087, 0.001114009, -0.017022725, -0.002176074, 0.018818496, 0.0035767017, 0.030186769, 0.00053149636, 0.004359569, -0.017408593, 0.01682979, 0.00065347156, -0.0081997, 0.0055728275, 0.021712508, -0.013052735, 0.0023875597, 0.0046378393, 0.006411349, -0.011947074, 0.015405047, -0.00033346063, 0.011264384, -0.0010017733, 0.009631865, 0.0106930025, -0.02282559, -0.0006989224, -0.020718155, -0.00036801255, 0.012392307, -0.011538944, 0.0024858818, -0.008652353, -0.013594435, -0.010641058, -0.008785923, 0.029993834, -0.010314555, -0.008229382, -0.014024826, -0.008489101, 0.0013932069, -0.0314631, -0.017423434, 0.025719602, -0.004452326, -0.014848506, -0.031374056, -0.030661684, 0.02021356, 0.00063028233, 0.0073055243, -0.013149202, -0.007643159, 0.008110654, 0.017868668, 0.0050942027, -0.0043410175, -0.0076357387, -0.0013069431, -0.037518263, -0.008600409, 0.001558314, 0.0030257264, -0.014670413, 0.014336489, 0.0013208566, -0.0006094121, -0.008785923, -0.0038698132, -0.015360524, 0.0076208976, 0.010062256, 0.005235193, 0.00711259, -0.013958041, 0.009194053, 0.0060885553, -0.0035878324, -0.012266157, -0.006904815, -0.013290192, 0.0053613423, 0.005109044, -0.017853826, 0.006537498, -0.0145442635, 0.0095725, -0.00097116356, -0.0111679165, 0.00020452871, -0.003144455, 0.001114009, -0.015137907, 0.0026713954, -0.0050199972, 0.010878515, 0.0080438685, 0.0064039286, 0.0028921566, 0.025274368, -0.018996589, 4.756684e-05, -0.00022934115, -0.010129041, -0.011516683, -0.008273905, -0.019664438, 0.0065263673, -0.012607502, -0.002951521, -0.0071830857, -0.015241795, 0.0058585186, -0.0010091938, -0.0041221115, 0.0061182375, -0.02801997, -0.015464411, 0.015063702, 0.0004939299, -0.0037881872, 0.016280672, -0.03270975, 0.015508935, -0.015212113, 0.031077234, -0.00860783, 0.016340036, 0.0024951575, -0.019219207, 0.009802537, 0.002923694, -0.051201746, 0.0018421498, -0.002406111, 0.02242488, -0.010403601, -0.011294066, -0.0010963852, -0.00806613, 0.0042222887, -0.001229955, 0.0097134905, -0.017067248, -0.0030943663, -0.01585028, 0.024992388, 0.01872945, -0.015924485, -0.0040441956, -0.023463756, -0.0059846677, -0.0077396263, 0.007205347, 0.000744837, -0.0031908334, 0.017527321, -0.008273905, 0.0038327104, -0.004125822, -0.002830937, -0.009490875, -0.007027254, 0.004389251, 0.0014627745, 0.0073352065, -0.004055327, 0.0144329555, -0.017735098, -0.015330842, 0.027708307, -3.0638726e-05, -0.008266484, -0.0069938614, 0.020569745, -0.019159842, 0.011679934, -0.014818824, 0.007097749, -0.016518129, 0.0042853635, 0.013253089, 0.0013848589, 0.013074996, 0.0056544538, -0.014314227, -0.010997244, -0.001981285, -0.0058585186, 0.025927376, -0.016518129, -0.0011863593, -0.018536516, 0.005506043, -0.024413586, 0.018210012, 0.021697666, 0.010915618, -0.00670075, -0.015597981, 0.020614268, 0.011175337, 0.0006390943, 0.008719138, -0.009008539, 0.01913016, 0.0039180466, 0.010893357, -0.0049272403, 0.02856909, 0.008489101, 0.0124739325, -0.015108225, -0.007301814, 0.022276469, -0.015345682, -0.003556295, -0.0154495705, 0.009668968, -0.002281817, -0.019026272, 0.00888981, -0.017527321, -0.014522002, 0.009936106, -0.016532969, -0.001257782, -0.022766225, -0.0043113353, -0.002584204, 0.00056674395, 0.012355204, 0.0058177058, 0.0042037377, 0.0172305, 0.024220651, -0.013609275, 0.008095812, 0.006518947, -0.022810748, -0.0072276085, -0.021386005, -0.020851726, 0.0013227117, 0.03291753, -0.011561206, -0.027292756, 0.00861525, 0.018521674, -0.009520557, 0.0015379075, -0.008837867, 0.010789469, -0.016043214, 0.01682979, 0.012347783, 0.003539599, -0.00085429, 0.0033670713, 0.0131269405, -0.020732997, -0.0012243895, 0.002009112, -0.0147965625, 0.003224226, -0.013060155, 0.019931577, 0.010010312, 0.011078871, -0.008029027, -0.027634101, 0.015100804, -0.012652026, -0.011286645, 0.018402945, 0.01366864, 0.014737198, 0.0032910106, -0.023552803, -0.0048159324, -0.009416669, 0.008511363, 0.005005156, 0.00074761966, 0.0015397626, 0.0020035466, -0.0061553405, 0.009631865, 0.009476033, -0.008385213, 0.015271477, -0.013906097, -0.03775572, 0.003213095, -0.038379047, -0.010974983, -0.011234702, 0.00943893, 4.7190013e-05, 0.011932232, -0.011412795, 0.005825126, -0.015642505, 0.010618797, 0.008058709, 0.002897722, -0.004456036, -0.008184859, 0.017735098, 0.017868668, 0.0074353837, 0.017349228, -0.035054643, -0.0017067249, -0.01763121, 0.020035466, 0.005031128, 0.00212042, -0.008711717, 0.008592988, 0.011323748, 0.00023432683, 0.009787696, 0.0033633611, 0.010195826, -0.013831892, -0.004018224, -0.001367235, 0.0041888966, 0.004622998, -0.004678652, -0.035203055, 0.0060106395, -0.0047120447, 0.0015035875, -0.005647033, 0.02981574, 0.007546692, -0.009550239, 0.01901143, 0.018120965, 0.0014414404, -0.0048456145, 0.0019905607, 0.002569363, -0.0006381667, 0.025808647, -0.0061887326, -0.008192279, 0.0039440184, 0.0067267222, -0.016636858, 0.0012549993, -0.003088801, -0.01435133, 0.0014683399, 0.006207284, 0.024458108, 0.018491993, -0.011702196, -0.017334389, 0.012184531, 0.00519438, 0.0035989631, -0.023834784, 0.008273905, -0.0031555858, -0.0019200655, -0.0043002046, -0.004663811, 0.0005509753, -0.0018885281, -0.01774994, -0.0024636202, -0.0030739598, 0.0060477424, 0.010514909, 0.0059920885, -0.025467303, -0.007261001, 0.013898676, 0.0107672075, -0.007813832, -0.005153567, -0.0068973945, -0.015939325, 0.0044894284, -0.005272296, -0.012548137, -0.0057063974, -0.003131469, 0.048975583, -0.001292102, 0.004841904, 0.01297853, -0.01819517, 0.0021000137, -0.0043929615, -0.034282908, 0.04087235, 0.012600082, 0.03707303, 0.02037681, 0.0095725, 0.0055839582, -0.0018338016, -0.018744292, -0.036746528, 0.01257782, 0.016458765, -0.021371163, -0.008177438, -0.008533624, -0.011776402, 0.011924813, 0.010893357, -0.008704296, -0.0055431454, 0.012971109, -0.016770426, 0.005647033, 0.006326013, -0.018239694, -0.006845451, -0.02883623, -0.008741399, -0.0026213068, -0.000302619, 0.006537498, 0.005316819, 0.0011065884, -0.027396644, -9.964107e-06, 0.021296958, -0.0060885553, 0.0038030283, -0.020154195, -0.013408921, -0.02187576, 0.0073723095, 0.0051609874, -0.0061442093, -0.008511363, -0.013045315, -0.015627664, 0.0062963306, -0.004114691, -0.0063927975, 0.027159186, -0.01898175, 0.013720583, 0.02267718, 0.00014191789, -0.014247443, 0.017319547, -0.0006488337, 0.014032247, 0.02968217, -0.0032372118, 0.0024302278, -0.0010091938, -0.0021296958, -0.0037362434, 0.025467303, 0.0020554904, -0.00025577683, 0.0037844768, 0.011197599, 0.002667685, -0.0021538124, 0.008296167, -0.0124368295, -0.012332942, 0.000985077, 0.009765434, -0.01708209, -0.024992388, -0.0071830857, 0.013579593, -0.0013811486, 0.017156295, 0.008451998, 0.0063816668, -0.030023517, -0.008014186, 0.0011307052, 0.008837867, 0.017037567, 0.009134688, 0.0032335015, 0.0002891693, 0.008266484, 0.0025916246, 0.025734443, 0.01693368, 0.017660892, 0.03143342, -0.0062295455, 0.020258082, 0.0005027418, -0.0001313668, 0.0041740555, -0.0011724457, -0.0031110626, -0.011642831, -0.035707653, -0.01026261, -0.005387314, -0.0056952666, -0.030988187, -0.01216969, -0.0029867685, 0.012518455, 0.0045154, -0.0036824443, -0.0068417406, -0.013156623, 0.010952721, 0.0017688719, -0.013023052, -0.007962243, -0.019456664, 0.02037681, 0.0054392577, -0.010091938, 0.023701213, -0.021133706, 0.018135807, 0.0032019643, -0.0058139954, -0.01598385, 0.0065820212, 0.0020220978, 0.024398744, 0.006737853, 0.01462589, 0.0130898375, -0.00020986222, -0.007858355, 0.020391652, 0.003335534, -0.039180465, -0.0054244166, -0.0074168327, -0.007628318, 0.00970607, -0.017260183, -0.006574601, -0.0134163415, -0.005494912, 0.0050014458, -0.005168408, 0.017957713, 0.014462638, 0.019560551, 0.002543391, -0.033392444, -0.022929477, 0.016711062, 0.014291965, 0.00917179, 0.0075058793, -0.0005333515, 0.021267276, -0.013275351, 0.0007049516, -0.013460864, 0.002200191, -0.02134148, -0.0029496658, 0.0100399945, -0.011145655, -0.0071571134, 0.014989496, 0.008221962, 0.0027938343, 0.0012642749, -0.0033986086, -0.019887054, -0.013906097, -0.022306152, -0.006715591, 0.013045315, -0.013631538, -0.022469403, -0.013082417, 0.0027140635, -0.012184531, 0.010425863, 0.0100399945, 0.0072127674, 0.0020295184, -0.0005277861, 0.0072016367, 0.008719138, -0.018655244, -0.0061553405, -0.0014970945, -0.012592661, 0.013423762, 0.0032817351, -0.007049516, -0.021786714, -0.0025174192, -0.014314227, -0.017037567, -4.6994355e-06, 0.022320993, -0.0029867685, 0.021222752, 0.028242586, 0.026713954, -2.2913824e-05, -0.008273905, -0.014959814, -0.014566526, -0.0043855407, 0.0015416178, 0.0029719274, 0.008555885, -0.017942872, -0.027025618, 0.005209221, -0.0031110626, 0.0193973, -0.010336816, 0.0073871505, -0.0032928658, 0.00082321645, -0.00085382623, -0.029192416, 0.011145655, -0.0065708905, 0.020465856, -0.008348111, 0.0005978175, -0.021549257, -0.010403601, 0.0020814622, -0.009208893, 0.010722685, -0.0031203383, 0.0015638794, -0.0006135861, 0.0022354384, -0.0010926749, -0.00016835357, -0.0037826216, -0.017794462, -0.027292756, 0.0063519846, -0.008719138, -0.033184666, 0.01311952, 0.008882389, -0.00081811484, 0.015390206, 0.0031277589, 0.0069270767, -0.012748493, 0.010782049, -0.03734017, -0.02062911, -0.020836884, -0.009090165, 0.023226298, -0.0063445643, -0.0007703451, 0.021207912, -0.012882062, 0.011153076, -0.013349556, -0.012540718, -0.0018514254, 0.00052175694, -0.027426327, 0.0051721185, 0.009691229, -0.0029218388, 0.0017762925, -0.0066896193, 0.0056507434, 0.011657673, 0.004841904, 0.008503942, -0.007068067, 0.033956405, -0.006478134, -0.016354876, 0.0037974627, 0.009424089, 0.01490787, -0.035885744, -0.00943151, 0.0065857316, -0.010232928, -0.0093127815, -0.0021556676, -0.011553785, 0.021460209, -0.00478625, -0.020955613, -0.008311008, 0.0057768924, 0.038646188, -0.0056099305, -0.010062256, -0.008793343, 0.0023189196, -0.018684927, 0.0059846677, 0.035054643, 0.03309562, -0.006177602, -0.005086782, 0.0042853635, 0.021964807, -0.013505388, 0.0052129314, 0.015627664, 0.015026599, 0.017824143, -0.0023857045, 0.013067576, 0.01339408, -0.019768326, 0.00048558178, -0.0051424364, 0.014061929, 0.014099032, 0.011442477, 0.020124512, 0.0037844768, 0.001251289, 0.00028058927, 0.0022577, -0.0019905607, -0.013520229, -0.028465202, -0.0017169281, 0.020228399, -0.004196317, -0.0061442093, -0.0038920748, 0.010537171, -0.003060974, -0.032205157, -0.0037881872, -0.0066413856, -0.015375365, 0.032442614, -0.010759787, 0.036746528, -0.010514909, -0.009483454, 0.02542278, 0.016161943, 0.008541045, 0.0128004365, -0.0015564589, 0.0017187832, 0.004077588, -0.010596535, -0.014395853, 0.005179539, 0.011598309, -0.006099686, 0.0026268722, 0.008785923, 0.007902878, 0.009965789, -0.011768981, 0.00048279908, 0.016562652, 0.0072387396, -0.0046007363, 0.018625563, 0.007917719, 0.01462589, -0.007902878, 0.001120502, -0.026580384, -0.034787506, -0.006604283, 0.0066005727, 0.0058214157, -0.008518783, -0.0029218388, -0.007431674, 0.00629262, -0.030513272, 0.0021148548, -0.017853826, -0.016147101, -0.006986441, -0.013497967, -0.0059698266, 0.01258524, -0.007665421, -0.016918838, -0.011190179, 0.008132915, -0.003472814, 0.009305361, -0.009899004, 0.003582267, 0.010574274, 0.014277125, -0.022231946, 0.02732244, 0.02475493, 0.0070087025, -0.020228399, -0.0066228346, 0.012971109, 0.035321783, 0.024428427, -0.009594762, -0.0051350156, 0.004070168, 0.0016130405, 0.014358751, 0.006441031, -0.0002798936, 0.005698977, -0.00052175694, -0.007331496, 0.009958369, -0.003266894, -0.016236147, -0.014455217, 0.009973209, -0.009743173, 0.016147101, -0.012674287, 0.0039328877, -0.009683808, -0.0056396127, -0.020673633, -0.024027718, 0.025734443, 0.012154849, -0.008860128, -0.010151302, -0.0069085252, -0.011553785, -0.0075986357, 5.2088733e-05, 0.008926913, -0.022944318, -0.029964153, 0.0037176919, 0.0003636066, 0.0126891285, 0.016325194, 0.020436175, -0.007776729, 0.017245341, 0.0045896056, 0.009053063, -0.010359078, 0.022320993, 0.021118864, -0.01353507, -0.0063816668, -0.019634755, -0.0061553405, -0.008392634, -0.023849625, -0.025778966, -0.014329068, 0.00012916382, 0.02474009, -0.02132664, -0.022039013, -0.008674614, -0.010544592, 0.016651697, -0.033244032, 0.0077247852, -0.0017976265, 0.02870266, 0.02147505, 0.028717501, 0.0018987313, -0.019798009, 0.008273905, -0.008503942, -0.014291965, -0.0018273087, -0.014855927, -0.015612822, -0.00091272674, 0.0032483425, 0.0075949254, -0.01271139, -0.02870266, 0.010225508, -0.024280015, -0.019872213, -0.0060774246, -0.002159378, -0.004155504, -0.017838985, -0.02024324, 0.004047906, 0.00369172, -0.0147965625, 0.003131469, -0.008503942, -0.02612031, -0.0028940118, 0.0029014323, -0.011390533, -0.012637184, 0.00095446734, 0.006504106, 0.0016390124, -0.011108552, -0.031225644, 0.0005435548, 0.010180985, -0.0034023188, 0.0042074476, 0.011397954, 0.025927376, -0.010648479, -0.020421334, 0.0121103255, -0.002476606, 0.008140336, -0.00629262, 0.008303587, 0.014603628, 0.030542955, 0.010180985, -0.0048085116, -0.014477479, 0.016903996, 0.010470386, -0.0058288365, -0.0043818303, -0.019426981, -0.0154495705, -0.01599869, 0.0020406493, -0.016221307, -0.0048307735, 0.002202046, 0.008095812, -0.01299337, -0.0060106395, 0.010381339, 0.008778502, 0.020540062, 0.0028884462, -0.0004681899, -0.008600409, -0.001886673, 0.010611376, -0.006448452, -0.0106930025, -0.020332288, -0.0065560495, 0.004841904, 0.009936106, 0.005506043, -0.009498294, 0.0029199836, 0.0041740555, -0.0022948028, -0.011947074, -0.0038178694, -0.0012410858, 0.016310353, 0.0037974627, 0.02021356, -0.00072211155, 0.017824143, 0.014729777, -0.006589442, -0.01311952, -0.0028383576, 0.00030401038, -0.015019178, 0.013683481, -0.00038238984, 0.015419888, 0.0031296138, -0.020896249, 0.011961915, 0.019723803, -0.032858163, 0.026743636, 0.015508935, -0.0007253581, 0.014136134, 0.016250988, 0.020940771, 0.004890138, -0.01791319, -0.009861901, -0.002923694, -0.017319547, -0.02092593, -0.014522002, 0.03582638, 0.0031926886, 0.016028373, 0.02187576, 0.0068009277, 0.004897558, -0.0180616, 0.015776074, 0.0016492156, 0.0026713954, -0.0031963987, -0.011323748, 0.005847388, 0.0055950894, -0.0044745873, 0.010373919, -0.02010967, -0.019857373, 0.015286318, -0.0067712455, 0.011323748, -0.002476606, 0.023612168, 0.017438276, -0.016711062, 0.020970454, 0.0068899738, -0.0067712455, -0.003144455, -0.017512482, 0.012778175, 0.02953376, 0.008266484, -0.010336816, 0.008964016, -0.003474669, 0.0051980903, 0.0011409085, 0.010566853, -0.01175414, -0.011405374, 0.021074342, 0.011435056, 0.019174682, -0.011798663, 0.009201473, 0.01024777, -0.010967562, -0.007097749, 0.015909644, -0.0010889646, 0.0029997544, 0.01693368, 0.0006979948, 0.024176128, -0.011679934, 0.024087083, 0.02733728, 0.0010212522, 0.0044263536, -0.011605729, 0.015597981, 0.0035785567, 0.009283099, -0.0037121265, -0.016013531, 0.02091109, 0.01926373, -0.00046703042, 0.021964807, -0.008385213, -0.02750053, 0.019575393, -0.0027029328, -0.014692674, -0.0020462146, 0.025912536, 0.0021909152, -0.011279225, 0.02720371, -0.0050199972, -0.012355204, 0.028910434, -0.0021742191, 0.0060180603, -0.0072164778, 0.01587996, -0.0041072704, -0.006107107, -0.004007093, 0.011442477, -0.0100399945, -0.00041647797, -0.008296167, 0.007164534, -0.011598309, 0.002775283, -0.008852707, -0.030231291, 0.010744946, -0.0007253581, 0.0032613287, -0.011323748, 0.02747085, -0.007873196, 0.017675733, -0.0026472786, -0.009179211, -0.0019219206, 0.013023052, 0.008288747, -0.022914637, -0.01721566, 0.009320201, 0.002226163, -0.00051201746, 0.018922385, -0.0043558585, -0.023448916, -0.011731878, -0.01763121, 0.02612031, 0.0025471014, -0.007947401, -0.024220651, 0.0014776156, -0.029014323, -0.0018273087, 0.00032998226, -0.0086968765, -0.008555885, -0.007353758, -0.016488446, 0.0007912154, -0.0134534445, 0.0063445643, 0.0028086754, 0.018892702, -0.0049643433, -0.0013996998, 0.014336489, 0.013720583, -0.022039013, 0.026832683, 0.008437158, 0.011768981, -0.01408419, -0.004619288, 0.028405838, 0.015568299, -0.00033392443, -0.001721566, 0.007695103, 0.013854153, 0.01996126, -0.0018681217, -0.03870555, -0.008644932, 0.016443923, -0.013045315, -0.02257329, -0.014729777, -0.0043261764, -0.0010574274, 0.003921757, 0.007776729, 0.003116628, -0.035470195, -0.009817378, 0.01313436, 0.016458765, -0.010878515, -0.00327988, 0.01571671, 0.01134601, 0.0103293955, -0.013468285, -0.0118209245, -0.0063779564, -0.023612168, -0.015597981, -0.0072127674, 0.0072238985, 0.0038327104, -0.011702196, 0.012050961, 0.0026546991, -0.01789835, 0.014262283, 0.009735752, 0.007439094, -0.02392383, -0.015597981, -0.008459419, 0.0042705224, -0.007665421, -0.033956405, 0.013891256, -0.0064595826, 0.0041221115, -0.0036991406, -0.006326013, 0.049123995, 0.0060551628, 0.0114202155, -0.004125822, 0.023048205, 0.0065078163, 0.014240022, -0.029415032, -0.0134015, 0.019812848, -0.019055953, -0.0039959624, -0.008236802, 0.016696222, -0.01093788, 0.009142109, -0.012666866, -0.0064447415, -0.008852707, -0.011412795, -0.022781067, -0.0045562135, 0.0014061929, 0.005943855, -0.0055468557, -0.0062852, -0.018744292, 0.003294721, 0.011806084, -0.0336299, 0.019471504, 0.0034134497, 0.0128004365, 0.012644605, -0.0009145819, 0.013245669, -0.028732343, 0.016666539, 0.003747374, -0.007431674, -0.011947074, -0.025125958, -0.005350211, -0.009913845, -0.017868668, -0.0009363798, -0.008051289, -0.00084315916, 0.007097749, 0.005554276, 0.014381012, -0.0014906016, -0.01789835, 0.005398445, 0.012125167, -0.0055468557, -0.0048085116, -0.008526203, 0.0010769062, -0.005947565, 0.0193973, -0.015597981, -0.0106930025, -0.012555558, -0.030839777, 0.018388106, -0.013868994, -0.0120732235, -0.0054169963, 0.04375152, 0.016221307, -0.00601435, -0.021964807, -0.009720911, 0.017022725, -6.974151e-05, -0.006667358, -0.00710888, -0.01819517, -0.030246133, -0.00683803, 0.009194053, 0.010848833, -0.008118073, -0.0014534988, -0.005238903, 0.0071163005, 0.0003923612, 0.019501187, 0.0038438411, -0.0017475379, 0.01803192, -0.004968053, -0.008214541, 0.0038512617, -0.010856254, 0.0003923612, 0.001476688, 0.0073648887, 0.015078543, -0.00410356, 0.015167589, 0.002066621, -0.0037269676, 0.005732369, 0.0014414404, -0.006619124, -0.018210012, -0.01681495, 0.009631865, 0.0008663484, -0.0033169826, -0.020762678, -0.0031537306, -0.001270768, -0.01093046, 0.0070866183, -0.0014776156, -0.0068046376, 0.005966116, 0.0038846543, -0.009965789, 0.017200818, -0.011130814, 0.009045642, -0.0028736051, 0.017289864, -0.00012800437, 0.01052975, 0.0013746555, -0.0134534445, -0.0042074476, -0.004352148, 0.026506178, -0.010611376, 0.01598385, 0.01834358, 0.0055320146, 0.0036787342, 0.021564098, 0.0010871096, 0.015071122, 0.004578475, -0.010559432, -0.017126612, 0.009409248, 0.02981574, 0.009802537, 0.0238051, -0.025482144, -0.010136462, -0.02310757, 0.005561697, 0.015657345, -0.0008811895, 0.005795444, 0.015538617, 0.010292293, 0.008459419, 0.026016423, -0.0028457781, 0.00085846405, 0.001762379, 0.013379239, 0.00055561314, 0.024799455, 0.016651697, -0.003732533, 0.003049843, 0.02257329, -0.008845287, 0.0063631153, 0.024443269, -0.0063631153, 0.0042705224, 0.0113831125, 0.009424089, 0.018625563, -0.01093788, 0.0075244303, -0.0030795252, 0.010032574, -0.0024153867, -0.0057880236, 0.020079989, -0.0064187697, -0.013186305, 0.0013087982, 0.027411485, 0.023448916, -0.01422518, -0.025111116, -0.019486345, 0.0077173645, 0.029578283, -0.01818033, -0.0011279226, 0.014262283, 0.017838985, 0.01654781, -0.0019627335, -0.02392383, -0.018373264, 0.0010991679]

In [60]:
len(em2)

3072

In [70]:
from openai import OpenAI
client = OpenAI(api_key='')



def generate_openai_embeddings(text):

    response = client.embeddings.create(
            model="text-embedding-3-small",
            input=text,
            encoding_format="float"
            )
    return response.data[0].embedding
embed = generate_openai_embeddings("Delta Lake, Parquet, and Spark SQL")
print(embed)

[0.0010368907, -0.024985319, 0.035952207, -0.029342758, 0.043254584, -0.012026266, -0.017269852, 0.023359608, -0.006239667, -0.022653356, 0.039630048, -0.008008627, 0.024545576, 0.006922599, -0.017083295, 0.03251423, 0.01535098, -2.229159e-05, -0.012419369, 0.07755443, -0.060870897, -0.012259463, -0.029529314, -0.027224002, -0.0035512466, 0.05407489, -0.024958668, 0.038084287, 0.010773669, -0.05005059, 0.0082284985, -0.03653853, -0.01840252, -0.01680346, 0.0136053385, 0.007355678, 0.017269852, 0.018442497, -0.032887343, -0.016816786, 0.017149923, 0.013485409, 0.00034937804, 0.023173051, 0.060764294, 0.028409973, -0.011366654, -0.043041375, -0.003784443, 0.02610466, -0.057299662, 0.019308655, 0.0062996317, -0.02394593, -0.049410965, 0.018602403, -0.0129257385, -0.010740356, 0.06284307, -0.03677839, 0.027983557, 0.01970842, 0.018202638, 0.079579905, -0.008688228, 0.025318457, -0.042215195, -0.029369408, -0.0088681225, 0.0068060006, -0.054154847, 0.026904192, 0.006855971, -0.043174632, 0.

In [71]:
from neo4j import GraphDatabase

def find_similar_nodes(description_embedding):
    # Connect to Neo4j database
    uri = "bolt://localhost:7687"
    user = "neo4j"
    password = "admin123"
    driver = GraphDatabase.driver(uri, auth=(user, password))

    # Define Cypher query
    query = """
    CALL db.index.vector.queryNodes('description_embed', 6, $embedding)
    YIELD node, score
    RETURN node.id AS id, score
    ORDER BY score DESC
    """

    # Execute the query
    similar_node_ids = []
    with driver.session() as session:
        result = session.run(query, embedding=description_embedding)
        for record in result:
            similar_node_ids.append(record["id"])

    # Close the Neo4j driver    
    driver.close()

    return similar_node_ids

# Example usage


similar_node_ids = find_similar_nodes(embed)
print("Similar node IDs:", similar_node_ids)


Similar node IDs: ['pmy_cv', 'p08652', 'p08134', 'p06119', 'p07561', 'p07207']


In [22]:
import cohere


Failed to write data to connection IPv4Address(('localhost', 7687)) (ResolvedIPv4Address(('127.0.0.1', 7687)))
Failed to write data to connection IPv4Address(('localhost', 7687)) (ResolvedIPv4Address(('127.0.0.1', 7687)))
/Users/thejas/Library/CloudStorage/OneDrive-NortheasternUniversity/Spring Semester 2024/GenAI with Graph/.graphml_resume/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [24]:
from llama_index.embeddings.cohere import CohereEmbedding
embed_model = CohereEmbedding(
    cohere_api_key="",
    model_name="embed-english-v3.0", # Supports all Cohere embed models
    input_type="search_query", # Required for v3 models
)
 
# Generate Embeddings
embeddings = embed_model.get_text_embedding("We're seeking a Front-end Developer with over three years of experience to spearhead the development of sophisticated web applications tailored for large enterprises. Your role will involve leading cross-functional teams in translating business requirements into seamless user experiences. You'll architect scalable front-end components using HTML, CSS, and JavaScript frameworks like React.js or Angular, ensuring optimal performance and accessibility across browsers and devices. Your expertise will drive the adoption of best practices and innovative technologies, while your mentorship will nurture a culture of continuous learning. If you're passionate about crafting exceptional user experiences and thrive in a dynamic, collaborative environment, we'd love to hear from you.")
 
# Print embeddings
print(len(embeddings))
print(type(embeddings[1]))
print(embeddings)

1024
<class 'float'>
[-0.0011634827, 0.0028743744, -0.045715332, -0.12536621, 0.00016152859, 0.006160736, -0.087890625, 0.04623413, 0.016464233, 0.006198883, -0.039001465, -0.0073928833, 0.0012130737, -0.024520874, 0.04043579, -0.03375244, 0.02217102, 0.01751709, 0.035980225, -0.014129639, 0.01234436, -0.0020160675, -0.038726807, 0.023239136, 0.06549072, 0.0014743805, -0.091674805, 0.002298355, 0.045715332, 0.052764893, 0.018554688, -0.026641846, 0.030349731, -0.00040388107, -0.010696411, -0.02418518, 0.019088745, -0.012229919, -0.0087890625, 0.014678955, 0.039123535, -0.013191223, -0.021957397, 0.0041007996, -0.04043579, -0.03353882, 0.04534912, -0.006324768, 0.025375366, -0.07342529, -0.026626587, -0.032836914, 0.0007762909, -0.07330322, -0.030838013, 0.020523071, -0.030227661, -0.021942139, 0.020263672, 0.013183594, -0.0057411194, -0.011421204, 0.024093628, -0.04824829, -0.0009880066, -0.010269165, -0.0038967133, 0.023269653, 0.06112671, 0.01436615, -0.06439209, 0.016448975, -0.0056

In [25]:
from openai import OpenAI
client = OpenAI(api_key='')



def generate_openai_embeddings(text):

    response = client.embeddings.create(
            model="text-embedding-3-small",
            input=text,
            encoding_format="float"
            )
    return response.data[0].embedding
emb = generate_openai_embeddings("Delta Lake, Parquet, and Spark SQL")


In [26]:
len(emb)

1536

In [27]:
from neo4j import GraphDatabase

def find_similar_nodes(description_embedding):
    # Connect to Neo4j database
    uri = "bolt://localhost:7689"
    user = "neo4j"
    password = "Pratikps1$"
    driver = GraphDatabase.driver(uri, auth=(user, password))

    # Define Cypher query
    query = """
    CALL db.index.vector.queryNodes('jobMatch', 6, $embedding)
    YIELD node, score
    RETURN node.id AS id, score
    ORDER BY score DESC
    """

    # Execute the query
    similar_node_ids = []
    with driver.session() as session:
        result = session.run(query, embedding=description_embedding)
        for record in result:
            similar_node_ids.append(record["id"])

    # Close the Neo4j driver    
    driver.close()

    return similar_node_ids

# Example usage


similar_node_ids = find_similar_nodes(emb)
print("Similar node IDs:", similar_node_ids)


Similar node IDs: ['p06275', 'p07001', 'p07151', 'p07756', 'p06067', 'p06952']
